In [ ]:
######################################################################################
# Using Electrodermal Activity to Detect Deception and Suspicion during a Card Game
# Affective Computing - Mini-project
# Jan Ondras
# Dec 2017 - Jan 2018
######################################################################################
# Testing of DECEPTION detection, person-independent manner
# using nested cross-validation: 
# outer loop for testing = LOSO, 
# inner loop for hyperparameter (delta_D, tau_MDEL, C) tuning = 5-fold
######################################################################################

# Nested CV links:
# https://stackoverflow.com/questions/42228735/scikit-learn-gridsearchcv-with-multiple-repetitions/42230764#42230764
# https://k-d-w.org/node/95
# https://sebastianraschka.com/faq/docs/evaluate-a-model.html
# https://chrisalbon.com/machine-learning/nested_cross_validation.html
# http://scikit-learn.org/stable/auto_examples/model_selection/plot_nested_cross_validation_iris.html
    
import itertools
import numpy as np
import glob
import csv
import time
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold, cross_val_score

seed1 = 23
seed2 = 889
seed3 = 43

 # To be used within GridSearch - inner CV
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed2)
# Outer CV - LOSO

int_SIDs = [1,2,3,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
SIDs = ['{:02d}'.format(s) for s in int_SIDs]
C_range = 2. ** np.arange(-30, 15, step=1.)
parameters = [{'C': C_range}]

eds = np.linspace(1.,4.,7) # Epoch delay
mels = np.linspace(0.5,4.5,9) # Max epoch length
feature_type = 'deception'
val_scores = []  # list of tuples (ed,mel,bestC,outer_fold, val_acc)   - outer_fold = SID 
test_scores = [] # list of tuples (ed,mel,outer_fold,test_acc)

for i, ed in enumerate(eds):
    for j, mel in enumerate(mels):
        t = time.time()
        data = np.loadtxt('./../Experiment/ExtractedFeatures/features_'+feature_type+'_u_'+str(i)+'_'+str(j)+'.csv', 
                          delimiter=",", skiprows=0, dtype=str)   
        SIDs_all = data[:,0]
        X = data[:,1:-2].astype(np.float32)
        y = data[:,-1].astype(np.int)

        # Outer CV - 2 min
        for SID in SIDs:
#             print "Subject ", SID, ": "
            X_test = np.array([features for subject, features in zip(SIDs_all, X) if subject == SID])
            y_test = np.array([label for subject, label in zip(SIDs_all, y) if subject == SID])
            X_train = np.array([features for subject, features in zip(SIDs_all, X) if subject != SID])
            y_train = np.array([label for subject, label in zip(SIDs_all, y) if subject != SID])
            # Balance the dataset
            X_train, y_train = RandomOverSampler(ratio='minority', random_state=seed1).fit_sample(X_train, y_train)
            X_test, y_test = RandomOverSampler(ratio='minority', random_state=seed1).fit_sample(X_test, y_test)
            # Inner CV
            grid = GridSearchCV(LinearSVC(), parameters, cv=inner_cv, n_jobs=8, scoring='accuracy')
            grid.fit(X_train, y_train)
            # Record validation score - used to select best ed, mel, C
            val_scores.append( (ed, mel, grid.best_params_['C'], SID, grid.best_score_) )
            # Using the best hyperparameters, train on full training set
            SVM_linear_best = grid.best_estimator_
            SVM_linear_best.fit(X_train, y_train)
            # Evaluate
            y_pred = SVM_linear_best.predict(X_test)
            # Record test score - for model evaluation only
            test_scores.append( (ed, mel, SID, accuracy_score(y_test, y_pred)) )
        
#         print ed, "\t", mel, ": \t", np.mean(r), "\t\t\t", time.time()-t
        print time.time()-t

# To save results
print val_scores
print test_scores

In [3]:
# Saved scores from above
val_scores=[(1.0, 0.5, 64.0, '01', 0.52331189710610937), (1.0, 0.5, 64.0, '02', 0.5192616372391653), (1.0, 0.5, 0.0009765625, '03', 0.52950558213716103), (1.0, 0.5, 2.0, '04', 0.52049180327868849), (1.0, 0.5, 64.0, '06', 0.53327922077922074), (1.0, 0.5, 0.0078125, '07', 0.5173913043478261), (1.0, 0.5, 32.0, '08', 0.51351351351351349), (1.0, 0.5, 64.0, '09', 0.51696284329563813), (1.0, 0.5, 0.0009765625, '10', 0.5106209150326797), (1.0, 0.5, 0.001953125, '11', 0.54631217838765012), (1.0, 0.5, 1.0, '12', 0.51916376306620204), (1.0, 0.5, 0.0625, '13', 0.53057851239669418), (1.0, 0.5, 32.0, '14', 0.52636054421768708), (1.0, 0.5, 32.0, '15', 0.52076411960132896), (1.0, 0.5, 0.0009765625, '16', 0.52684563758389258), (1.0, 0.5, 4.0, '17', 0.50501672240802675), (1.0, 0.5, 64.0, '18', 0.52570480928689889), (1.0, 0.5, 32.0, '19', 0.52809917355371905), (1.0, 0.5, 16.0, '20', 0.53403565640194495), (1.0, 1.0, 0.03125, '01', 0.547427652733119), (1.0, 1.0, 0.0625, '02', 0.5593900481540931), (1.0, 1.0, 0.03125, '03', 0.54864433811802238), (1.0, 1.0, 0.001953125, '04', 0.54426229508196722), (1.0, 1.0, 0.125, '06', 0.54383116883116878), (1.0, 1.0, 1.0, '07', 0.54347826086956519), (1.0, 1.0, 0.001953125, '08', 0.53153153153153154), (1.0, 1.0, 0.00390625, '09', 0.54200323101777059), (1.0, 1.0, 0.00390625, '10', 0.54493464052287577), (1.0, 1.0, 0.125, '11', 0.57032590051457976), (1.0, 1.0, 16.0, '12', 0.54878048780487809), (1.0, 1.0, 2.0, '13', 0.53966942148760333), (1.0, 1.0, 0.0625, '14', 0.56802721088435371), (1.0, 1.0, 0.03125, '15', 0.54817275747508309), (1.0, 1.0, 0.03125, '16', 0.53691275167785235), (1.0, 1.0, 0.0625, '17', 0.52759197324414719), (1.0, 1.0, 0.00390625, '18', 0.54394693200663347), (1.0, 1.0, 0.25, '19', 0.54132231404958675), (1.0, 1.0, 0.25, '20', 0.55915721231766613), (1.0, 1.5, 0.001953125, '01', 0.53778135048231512), (1.0, 1.5, 0.0625, '02', 0.5425361155698234), (1.0, 1.5, 0.015625, '03', 0.53907496012759171), (1.0, 1.5, 0.03125, '04', 0.53688524590163933), (1.0, 1.5, 0.25, '06', 0.5308441558441559), (1.0, 1.5, 0.25, '07', 0.53217391304347827), (1.0, 1.5, 0.00390625, '08', 0.55045045045045049), (1.0, 1.5, 0.001953125, '09', 0.54523424878836835), (1.0, 1.5, 0.00390625, '10', 0.55392156862745101), (1.0, 1.5, 0.0078125, '11', 0.56861063464837047), (1.0, 1.5, 0.015625, '12', 0.54790940766550522), (1.0, 1.5, 0.00390625, '13', 0.54049586776859504), (1.0, 1.5, 0.0078125, '14', 0.56547619047619047), (1.0, 1.5, 0.125, '15', 0.54983388704318936), (1.0, 1.5, 0.0078125, '16', 0.54949664429530198), (1.0, 1.5, 16.0, '17', 0.53846153846153844), (1.0, 1.5, 0.001953125, '18', 0.54477611940298509), (1.0, 1.5, 0.00390625, '19', 0.53966942148760333), (1.0, 1.5, 1.0, '20', 0.54457050243111826), (1.0, 2.0, 0.5, '01', 0.54662379421221863), (1.0, 2.0, 0.001953125, '02', 0.5393258426966292), (1.0, 2.0, 0.00048828125, '03', 0.54864433811802238), (1.0, 2.0, 0.00048828125, '04', 0.55163934426229511), (1.0, 2.0, 6.103515625e-05, '06', 0.54788961038961037), (1.0, 2.0, 0.000244140625, '07', 0.54956521739130437), (1.0, 2.0, 0.001953125, '08', 0.55315315315315317), (1.0, 2.0, 64.0, '09', 0.54765751211631664), (1.0, 2.0, 0.00390625, '10', 0.55800653594771243), (1.0, 2.0, 0.001953125, '11', 0.56775300171526588), (1.0, 2.0, 0.00390625, '12', 0.54703832752613235), (1.0, 2.0, 0.001953125, '13', 0.53966942148760333), (1.0, 2.0, 0.03125, '14', 0.57482993197278909), (1.0, 2.0, 0.25, '15', 0.54734219269102991), (1.0, 2.0, 0.0078125, '16', 0.5511744966442953), (1.0, 2.0, 0.0009765625, '17', 0.52675585284280935), (1.0, 2.0, 0.001953125, '18', 0.54394693200663347), (1.0, 2.0, 0.00048828125, '19', 0.54049586776859504), (1.0, 2.0, 0.00390625, '20', 0.54619124797406804), (1.0, 2.5, 0.5, '01', 0.547427652733119), (1.0, 2.5, 0.0009765625, '02', 0.5409309791332263), (1.0, 2.5, 0.001953125, '03', 0.54226475279106856), (1.0, 2.5, 0.5, '04', 0.53934426229508192), (1.0, 2.5, 0.5, '06', 0.53733766233766234), (1.0, 2.5, 128.0, '07', 0.54347826086956519), (1.0, 2.5, 0.125, '08', 0.56036036036036041), (1.0, 2.5, 0.00048828125, '09', 0.54361873990306941), (1.0, 2.5, 0.0009765625, '10', 0.54983660130718959), (1.0, 2.5, 0.015625, '11', 0.56260720411663812), (1.0, 2.5, 0.125, '12', 0.55662020905923348), (1.0, 2.5, 0.015625, '13', 0.54297520661157028), (1.0, 2.5, 0.25, '14', 0.57482993197278909), (1.0, 2.5, 0.25, '15', 0.55232558139534882), (1.0, 2.5, 0.25, '16', 0.55285234899328861), (1.0, 2.5, 4.0, '17', 0.52926421404682278), (1.0, 2.5, 0.001953125, '18', 0.54892205638474301), (1.0, 2.5, 64.0, '19', 0.54545454545454541), (1.0, 2.5, 0.0009765625, '20', 0.54457050243111826), (1.0, 3.0, 0.125, '01', 0.53456591639871387), (1.0, 3.0, 0.25, '02', 0.5377207062600321), (1.0, 3.0, 0.0625, '03', 0.53748006379585322), (1.0, 3.0, 32.0, '04', 0.53770491803278686), (1.0, 3.0, 0.5, '06', 0.5308441558441559), (1.0, 3.0, 32.0, '07', 0.53565217391304343), (1.0, 3.0, 0.00390625, '08', 0.55225225225225227), (1.0, 3.0, 0.000244140625, '09', 0.54442649434571888), (1.0, 3.0, 0.0001220703125, '10', 0.54166666666666663), (1.0, 3.0, 0.0078125, '11', 0.55145797598627788), (1.0, 3.0, 0.0625, '12', 0.54529616724738672), (1.0, 3.0, 0.5, '13', 0.55785123966942152), (1.0, 3.0, 0.03125, '14', 0.56887755102040816), (1.0, 3.0, 0.00390625, '15', 0.54734219269102991), (1.0, 3.0, 0.0625, '16', 0.54446308724832215), (1.0, 3.0, 8.0, '17', 0.53846153846153844), (1.0, 3.0, 32.0, '18', 0.54228855721393032), (1.0, 3.0, 4.0, '19', 0.53305785123966942), (1.0, 3.0, 0.00048828125, '20', 0.53970826580226905), (1.0, 3.5, 0.0078125, '01', 0.542604501607717), (1.0, 3.5, 0.000244140625, '02', 0.5425361155698234), (1.0, 3.5, 2.0, '03', 0.53827751196172247), (1.0, 3.5, 4.0, '04', 0.54262295081967216), (1.0, 3.5, 0.00048828125, '06', 0.54301948051948057), (1.0, 3.5, 1.0, '07', 0.55304347826086953), (1.0, 3.5, 0.0078125, '08', 0.55315315315315317), (1.0, 3.5, 4.76837158203125e-07, '09', 0.54442649434571888), (1.0, 3.5, 0.5, '10', 0.54493464052287577), (1.0, 3.5, 0.00390625, '11', 0.56260720411663812), (1.0, 3.5, 0.03125, '12', 0.55226480836236935), (1.0, 3.5, 0.25, '13', 0.54380165289256199), (1.0, 3.5, 0.0078125, '14', 0.56717687074829937), (1.0, 3.5, 0.0078125, '15', 0.54651162790697672), (1.0, 3.5, 0.0078125, '16', 0.55536912751677847), (1.0, 3.5, 0.00390625, '17', 0.54013377926421402), (1.0, 3.5, 0.015625, '18', 0.54809286898839138), (1.0, 3.5, 1.0, '19', 0.54793388429752066), (1.0, 3.5, 0.125, '20', 0.54700162074554293), (1.0, 4.0, 0.00390625, '01', 0.54662379421221863), (1.0, 4.0, 0.000244140625, '02', 0.5521669341894061), (1.0, 4.0, 0.00048828125, '03', 0.54146730462519932), (1.0, 4.0, 0.0078125, '04', 0.53524590163934427), (1.0, 4.0, 0.0009765625, '06', 0.5357142857142857), (1.0, 4.0, 1.0, '07', 0.54608695652173911), (1.0, 4.0, 0.0078125, '08', 0.5477477477477477), (1.0, 4.0, 1.52587890625e-05, '09', 0.54442649434571888), (1.0, 4.0, 0.0078125, '10', 0.54493464052287577), (1.0, 4.0, 0.0078125, '11', 0.56260720411663812), (1.0, 4.0, 0.015625, '12', 0.56968641114982577), (1.0, 4.0, 0.5, '13', 0.54628099173553724), (1.0, 4.0, 0.00390625, '14', 0.57057823129251706), (1.0, 4.0, 0.0078125, '15', 0.54983388704318936), (1.0, 4.0, 0.0009765625, '16', 0.54446308724832215), (1.0, 4.0, 0.25, '17', 0.53846153846153844), (1.0, 4.0, 0.015625, '18', 0.54892205638474301), (1.0, 4.0, 0.0078125, '19', 0.54545454545454541), (1.0, 4.0, 0.00048828125, '20', 0.55105348460291737), (1.0, 4.5, 32.0, '01', 0.54823151125401925), (1.0, 4.5, 0.000244140625, '02', 0.5497592295345104), (1.0, 4.5, 0.000244140625, '03', 0.54226475279106856), (1.0, 4.5, 0.015625, '04', 0.54180327868852463), (1.0, 4.5, 0.0001220703125, '06', 0.54301948051948057), (1.0, 4.5, 128.0, '07', 0.54869565217391303), (1.0, 4.5, 0.000244140625, '08', 0.5495495495495496), (1.0, 4.5, 0.000244140625, '09', 0.54281098546042006), (1.0, 4.5, 0.0078125, '10', 0.5490196078431373), (1.0, 4.5, 32.0, '11', 0.56603773584905659), (1.0, 4.5, 0.015625, '12', 0.56358885017421601), (1.0, 4.5, 2.0, '13', 0.55289256198347103), (1.0, 4.5, 0.015625, '14', 0.56717687074829937), (1.0, 4.5, 0.00390625, '15', 0.55232558139534882), (1.0, 4.5, 0.0625, '16', 0.5511744966442953), (1.0, 4.5, 0.00048828125, '17', 0.53595317725752512), (1.0, 4.5, 0.00390625, '18', 0.54892205638474301), (1.0, 4.5, 16.0, '19', 0.54876033057851237), (1.0, 4.5, 0.015625, '20', 0.55591572123176658), (1.5, 0.5, 4.0, '01', 0.53536977491961413), (1.5, 0.5, 0.125, '02', 0.5385232744783307), (1.5, 0.5, 64.0, '03', 0.53508771929824561), (1.5, 0.5, 0.0625, '04', 0.54344262295081969), (1.5, 0.5, 0.5, '06', 0.52029220779220775), (1.5, 0.5, 2.0, '07', 0.53913043478260869), (1.5, 0.5, 8.0, '08', 0.52522522522522519), (1.5, 0.5, 0.125, '09', 0.52746365105008075), (1.5, 0.5, 0.0625, '10', 0.51879084967320266), (1.5, 0.5, 32.0, '11', 0.54030874785591765), (1.5, 0.5, 16.0, '12', 0.51480836236933802), (1.5, 0.5, 0.015625, '13', 0.53636363636363638), (1.5, 0.5, 0.0625, '14', 0.53911564625850339), (1.5, 0.5, 8.0, '15', 0.53820598006644516), (1.5, 0.5, 0.125, '16', 0.54026845637583898), (1.5, 0.5, 0.0625, '17', 0.52173913043478259), (1.5, 0.5, 0.125, '18', 0.54311774461028195), (1.5, 0.5, 0.03125, '19', 0.53388429752066113), (1.5, 0.5, 0.0625, '20', 0.52998379254457051), (1.5, 1.0, 0.015625, '01', 0.545016077170418), (1.5, 1.0, 0.00390625, '02', 0.5473515248796148), (1.5, 1.0, 64.0, '03', 0.5470494417862839), (1.5, 1.0, 0.03125, '04', 0.55327868852459017), (1.5, 1.0, 0.03125, '06', 0.54626623376623373), (1.5, 1.0, 0.0625, '07', 0.55304347826086953), (1.5, 1.0, 0.0078125, '08', 0.536036036036036), (1.5, 1.0, 0.001953125, '09', 0.5395799676898223), (1.5, 1.0, 0.015625, '10', 0.55800653594771243), (1.5, 1.0, 0.125, '11', 0.55574614065180106), (1.5, 1.0, 0.00390625, '12', 0.54965156794425085), (1.5, 1.0, 0.00390625, '13', 0.55289256198347103), (1.5, 1.0, 0.001953125, '14', 0.5535714285714286), (1.5, 1.0, 0.00390625, '15', 0.55232558139534882), (1.5, 1.0, 0.00390625, '16', 0.54949664429530198), (1.5, 1.0, 0.0078125, '17', 0.54682274247491636), (1.5, 1.0, 0.03125, '18', 0.55721393034825872), (1.5, 1.0, 0.00390625, '19', 0.54958677685950408), (1.5, 1.0, 64.0, '20', 0.55753646677471635), (1.5, 1.5, 0.001953125, '01', 0.55707395498392287), (1.5, 1.5, 32.0, '02', 0.5585874799357945), (1.5, 1.5, 0.03125, '03', 0.55263157894736847), (1.5, 1.5, 0.0009765625, '04', 0.55245901639344264), (1.5, 1.5, 0.0009765625, '06', 0.55681818181818177), (1.5, 1.5, 32.0, '07', 0.54434782608695653), (1.5, 1.5, 0.00048828125, '08', 0.52792792792792798), (1.5, 1.5, 0.0009765625, '09', 0.55815831987075926), (1.5, 1.5, 64.0, '10', 0.53921568627450978), (1.5, 1.5, 0.0009765625, '11', 0.55831903945111494), (1.5, 1.5, 0.0078125, '12', 0.56010452961672474), (1.5, 1.5, 0.001953125, '13', 0.54710743801652895), (1.5, 1.5, 0.001953125, '14', 0.56122448979591832), (1.5, 1.5, 2.0, '15', 0.54485049833887045), (1.5, 1.5, 0.001953125, '16', 0.56291946308724827), (1.5, 1.5, 64.0, '17', 0.53929765886287628), (1.5, 1.5, 8.0, '18', 0.56301824212271978), (1.5, 1.5, 0.00390625, '19', 0.54710743801652895), (1.5, 1.5, 0.0009765625, '20', 0.55915721231766613), (1.5, 2.0, 0.0009765625, '01', 0.54099678456591638), (1.5, 2.0, 0.0009765625, '02', 0.5481540930979133), (1.5, 2.0, 0.001953125, '03', 0.5470494417862839), (1.5, 2.0, 0.0009765625, '04', 0.53278688524590168), (1.5, 2.0, 0.001953125, '06', 0.54301948051948057), (1.5, 2.0, 0.001953125, '07', 0.52869565217391301), (1.5, 2.0, 0.001953125, '08', 0.53243243243243243), (1.5, 2.0, 0.00048828125, '09', 0.54038772213247177), (1.5, 2.0, 3.0517578125e-05, '10', 0.52369281045751637), (1.5, 2.0, 0.03125, '11', 0.55831903945111494), (1.5, 2.0, 0.001953125, '12', 0.54268292682926833), (1.5, 2.0, 0.0009765625, '13', 0.54380165289256199), (1.5, 2.0, 0.001953125, '14', 0.56632653061224492), (1.5, 2.0, 0.001953125, '15', 0.54900332225913617), (1.5, 2.0, 0.0009765625, '16', 0.5511744966442953), (1.5, 2.0, 0.001953125, '17', 0.52341137123745818), (1.5, 2.0, 0.001953125, '18', 0.53897180762852404), (1.5, 2.0, 0.0009765625, '19', 0.54545454545454541), (1.5, 2.0, 0.001953125, '20', 0.55105348460291737), (1.5, 2.5, 0.001953125, '01', 0.54581993569131837), (1.5, 2.5, 0.0009765625, '02', 0.5545746388443018), (1.5, 2.5, 0.00390625, '03', 0.54944178628389151), (1.5, 2.5, 0.00048828125, '04', 0.53770491803278686), (1.5, 2.5, 32.0, '06', 0.54626623376623373), (1.5, 2.5, 0.03125, '07', 0.52260869565217394), (1.5, 2.5, 0.001953125, '08', 0.54234234234234235), (1.5, 2.5, 0.0009765625, '09', 0.5484652665589661), (1.5, 2.5, 0.0625, '10', 0.5220588235294118), (1.5, 2.5, 0.00390625, '11', 0.55660377358490565), (1.5, 2.5, 0.001953125, '12', 0.54878048780487809), (1.5, 2.5, 0.001953125, '13', 0.52892561983471076), (1.5, 2.5, 0.00390625, '14', 0.55867346938775508), (1.5, 2.5, 0.00390625, '15', 0.54734219269102991), (1.5, 2.5, 0.00390625, '16', 0.55453020134228193), (1.5, 2.5, 0.03125, '17', 0.52926421404682278), (1.5, 2.5, 0.001953125, '18', 0.54975124378109452), (1.5, 2.5, 0.00390625, '19', 0.54297520661157028), (1.5, 2.5, 0.00390625, '20', 0.55510534846029169), (1.5, 3.0, 0.0078125, '01', 0.53536977491961413), (1.5, 3.0, 0.00390625, '02', 0.5521669341894061), (1.5, 3.0, 0.0078125, '03', 0.53508771929824561), (1.5, 3.0, 0.0009765625, '04', 0.53278688524590168), (1.5, 3.0, 16.0, '06', 0.55925324675324672), (1.5, 3.0, 32.0, '07', 0.53565217391304343), (1.5, 3.0, 0.001953125, '08', 0.54504504504504503), (1.5, 3.0, 0.001953125, '09', 0.54442649434571888), (1.5, 3.0, 0.0078125, '10', 0.52124183006535951), (1.5, 3.0, 0.03125, '11', 0.55746140651801024), (1.5, 3.0, 0.001953125, '12', 0.55226480836236935), (1.5, 3.0, 0.001953125, '13', 0.52975206611570247), (1.5, 3.0, 0.0009765625, '14', 0.55612244897959184), (1.5, 3.0, 0.0078125, '15', 0.54900332225913617), (1.5, 3.0, 0.015625, '16', 0.54865771812080533), (1.5, 3.0, 0.03125, '17', 0.52759197324414719), (1.5, 3.0, 0.00390625, '18', 0.55472636815920395), (1.5, 3.0, 0.125, '19', 0.54710743801652895), (1.5, 3.0, 1.0, '20', 0.54619124797406804), (1.5, 3.5, 0.00390625, '01', 0.53456591639871387), (1.5, 3.5, 0.0078125, '02', 0.5425361155698234), (1.5, 3.5, 32.0, '03', 0.53429027113237637), (1.5, 3.5, 0.015625, '04', 0.5360655737704918), (1.5, 3.5, 16.0, '06', 0.5535714285714286), (1.5, 3.5, 0.125, '07', 0.52086956521739125), (1.5, 3.5, 0.00390625, '08', 0.53783783783783778), (1.5, 3.5, 0.00390625, '09', 0.55088852988691439), (1.5, 3.5, 128.0, '10', 0.52941176470588236), (1.5, 3.5, 0.00390625, '11', 0.55488850771869636), (1.5, 3.5, 0.00390625, '12', 0.54965156794425085), (1.5, 3.5, 16.0, '13', 0.52314049586776856), (1.5, 3.5, 0.03125, '14', 0.55782312925170063), (1.5, 3.5, 0.03125, '15', 0.54069767441860461), (1.5, 3.5, 64.0, '16', 0.54446308724832215), (1.5, 3.5, 0.03125, '17', 0.53846153846153844), (1.5, 3.5, 64.0, '18', 0.55472636815920395), (1.5, 3.5, 0.125, '19', 0.54628099173553724), (1.5, 3.5, 0.125, '20', 0.5486223662884927), (1.5, 4.0, 0.0078125, '01', 0.54019292604501612), (1.5, 4.0, 0.00390625, '02', 0.550561797752809), (1.5, 4.0, 0.0009765625, '03', 0.53349282296650713), (1.5, 4.0, 0.03125, '04', 0.53196721311475414), (1.5, 4.0, 0.25, '06', 0.53814935064935066), (1.5, 4.0, 0.0625, '07', 0.52521739130434786), (1.5, 4.0, 0.001953125, '08', 0.54594594594594592), (1.5, 4.0, 0.0078125, '09', 0.54119547657512113), (1.5, 4.0, 0.0009765625, '10', 0.53513071895424835), (1.5, 4.0, 0.001953125, '11', 0.565180102915952), (1.5, 4.0, 0.00390625, '12', 0.55836236933797911), (1.5, 4.0, 0.00390625, '13', 0.52148760330578514), (1.5, 4.0, 0.0078125, '14', 0.56292517006802723), (1.5, 4.0, 0.015625, '15', 0.53488372093023251), (1.5, 4.0, 0.00390625, '16', 0.5436241610738255), (1.5, 4.0, 0.03125, '17', 0.5376254180602007), (1.5, 4.0, 0.015625, '18', 0.55306799336650081), (1.5, 4.0, 128.0, '19', 0.54132231404958675), (1.5, 4.0, 0.00390625, '20', 0.5486223662884927), (1.5, 4.5, 0.001953125, '01', 0.53536977491961413), (1.5, 4.5, 0.00390625, '02', 0.5553772070626003), (1.5, 4.5, 0.00390625, '03', 0.53508771929824561), (1.5, 4.5, 0.00390625, '04', 0.52459016393442626), (1.5, 4.5, 0.0078125, '06', 0.52353896103896103), (1.5, 4.5, 0.001953125, '07', 0.52000000000000002), (1.5, 4.5, 0.001953125, '08', 0.55045045045045049), (1.5, 4.5, 0.0078125, '09', 0.54119547657512113), (1.5, 4.5, 0.001953125, '10', 0.53839869281045749), (1.5, 4.5, 0.001953125, '11', 0.56260720411663812), (1.5, 4.5, 0.00390625, '12', 0.56097560975609762), (1.5, 4.5, 0.001953125, '13', 0.51735537190082648), (1.5, 4.5, 0.00390625, '14', 0.56462585034013602), (1.5, 4.5, 0.00390625, '15', 0.54069767441860461), (1.5, 4.5, 0.0078125, '16', 0.55201342281879195), (1.5, 4.5, 0.015625, '17', 0.52090301003344486), (1.5, 4.5, 0.0078125, '18', 0.54975124378109452), (1.5, 4.5, 0.015625, '19', 0.54628099173553724), (1.5, 4.5, 0.0078125, '20', 0.54619124797406804), (2.0, 0.5, 512.0, '01', 0.50080385852090037), (2.0, 0.5, 1024.0, '02', 0.5016051364365971), (2.0, 0.5, 6.103515625e-05, '03', 0.50558213716108458), (2.0, 0.5, 128.0, '04', 0.50409836065573765), (2.0, 0.5, 512.0, '06', 0.50324675324675328), (2.0, 0.5, 0.000244140625, '07', 0.50260869565217392), (2.0, 0.5, 6.103515625e-05, '08', 0.50270270270270268), (2.0, 0.5, 64.0, '09', 0.52827140549273022), (2.0, 0.5, 0.0078125, '10', 0.50898692810457513), (2.0, 0.5, 16.0, '11', 0.53602058319039447), (2.0, 0.5, 0.000244140625, '12', 0.50087108013937287), (2.0, 0.5, 0.03125, '13', 0.50661157024793391), (2.0, 0.5, 128.0, '14', 0.50170068027210879), (2.0, 0.5, 0.000244140625, '15', 0.50083056478405319), (2.0, 0.5, 64.0, '16', 0.50671140939597314), (2.0, 0.5, 0.001953125, '17', 0.50418060200668902), (2.0, 0.5, 2048.0, '18', 0.50248756218905477), (2.0, 0.5, 128.0, '19', 0.50082644628099171), (2.0, 0.5, 32.0, '20', 0.51296596434359809), (2.0, 1.0, 1024.0, '01', 0.5088424437299035), (2.0, 1.0, 8.0, '02', 0.5056179775280899), (2.0, 1.0, 64.0, '03', 0.50637958532695371), (2.0, 1.0, 16.0, '04', 0.52131147540983602), (2.0, 1.0, 1024.0, '06', 0.50081168831168832), (2.0, 1.0, 1024.0, '07', 0.50956521739130434), (2.0, 1.0, 1024.0, '08', 0.50270270270270268), (2.0, 1.0, 0.00048828125, '09', 0.50484652665589658), (2.0, 1.0, 2.0, '10', 0.50326797385620914), (2.0, 1.0, 32.0, '11', 0.54202401372212694), (2.0, 1.0, 1024.0, '12', 0.50174216027874563), (2.0, 1.0, 64.0, '13', 0.51487603305785123), (2.0, 1.0, 8.0, '14', 0.53061224489795922), (2.0, 1.0, 8.0, '15', 0.50249169435215946), (2.0, 1.0, 256.0, '16', 0.50083892617449666), (2.0, 1.0, 32.0, '17', 0.51003344481605351), (2.0, 1.0, 16.0, '18', 0.5149253731343284), (2.0, 1.0, 32.0, '19', 0.5057851239669422), (2.0, 1.0, 1.0, '20', 0.51053484602917343), (2.0, 1.5, 0.00390625, '01', 0.52572347266881025), (2.0, 1.5, 0.0009765625, '02', 0.5232744783306581), (2.0, 1.5, 0.001953125, '03', 0.51196172248803828), (2.0, 1.5, 0.00048828125, '04', 0.5131147540983606), (2.0, 1.5, 0.0078125, '06', 0.50568181818181823), (2.0, 1.5, 0.00390625, '07', 0.51652173913043475), (2.0, 1.5, 0.0009765625, '08', 0.52072072072072073), (2.0, 1.5, 0.0078125, '09', 0.51857835218093695), (2.0, 1.5, 3.0517578125e-05, '10', 0.50081699346405228), (2.0, 1.5, 0.0078125, '11', 0.55145797598627788), (2.0, 1.5, 0.00390625, '12', 0.52961672473867594), (2.0, 1.5, 8.0, '13', 0.51735537190082648), (2.0, 1.5, 0.001953125, '14', 0.52210884353741494), (2.0, 1.5, 0.0078125, '15', 0.52408637873754149), (2.0, 1.5, 0.001953125, '16', 0.51845637583892612), (2.0, 1.5, 0.00390625, '17', 0.52341137123745818), (2.0, 1.5, 0.00390625, '18', 0.5265339966832504), (2.0, 1.5, 0.00390625, '19', 0.52066115702479343), (2.0, 1.5, 0.0009765625, '20', 0.51782820097244731), (2.0, 2.0, 0.00390625, '01', 0.542604501607717), (2.0, 2.0, 0.0078125, '02', 0.5337078651685393), (2.0, 2.0, 0.0009765625, '03', 0.5287081339712919), (2.0, 2.0, 0.00390625, '04', 0.52868852459016391), (2.0, 2.0, 0.001953125, '06', 0.51623376623376627), (2.0, 2.0, 0.0009765625, '07', 0.52695652173913043), (2.0, 2.0, 0.00048828125, '08', 0.52162162162162162), (2.0, 2.0, 0.0625, '09', 0.5266558966074314), (2.0, 2.0, 1.52587890625e-05, '10', 0.50653594771241828), (2.0, 2.0, 0.001953125, '11', 0.54716981132075471), (2.0, 2.0, 0.001953125, '12', 0.54006968641114983), (2.0, 2.0, 0.00390625, '13', 0.52892561983471076), (2.0, 2.0, 0.001953125, '14', 0.53316326530612246), (2.0, 2.0, 0.25, '15', 0.53820598006644516), (2.0, 2.0, 0.03125, '16', 0.53607382550335569), (2.0, 2.0, 0.001953125, '17', 0.53595317725752512), (2.0, 2.0, 1.0, '18', 0.53814262023217252), (2.0, 2.0, 0.0009765625, '19', 0.52975206611570247), (2.0, 2.0, 0.5, '20', 0.53808752025931927), (2.0, 2.5, 0.0009765625, '01', 0.52893890675241162), (2.0, 2.5, 0.0078125, '02', 0.5264847512038523), (2.0, 2.5, 0.000244140625, '03', 0.52312599681020733), (2.0, 2.5, 0.0078125, '04', 0.51393442622950825), (2.0, 2.5, 0.0001220703125, '06', 0.50730519480519476), (2.0, 2.5, 0.0078125, '07', 0.53565217391304343), (2.0, 2.5, 64.0, '08', 0.52522522522522519), (2.0, 2.5, 3.0517578125e-05, '09', 0.51130856219709209), (2.0, 2.5, 1.1920928955078125e-07, '10', 0.50326797385620914), (2.0, 2.5, 0.25, '11', 0.55231560891938247), (2.0, 2.5, 0.0009765625, '12', 0.53745644599303133), (2.0, 2.5, 0.0078125, '13', 0.52148760330578514), (2.0, 2.5, 8.0, '14', 0.54591836734693877), (2.0, 2.5, 0.0625, '15', 0.53986710963455153), (2.0, 2.5, 0.0009765625, '16', 0.53020134228187921), (2.0, 2.5, 0.001953125, '17', 0.52675585284280935), (2.0, 2.5, 4.0, '18', 0.53316749585406298), (2.0, 2.5, 0.0009765625, '19', 0.53471074380165284), (2.0, 2.5, 0.125, '20', 0.53727714748784439), (2.0, 3.0, 0.0078125, '01', 0.53295819935691313), (2.0, 3.0, 0.00048828125, '02', 0.5329052969502408), (2.0, 3.0, 0.000244140625, '03', 0.52312599681020733), (2.0, 3.0, 0.001953125, '04', 0.52049180327868849), (2.0, 3.0, 64.0, '06', 0.5178571428571429), (2.0, 3.0, 0.001953125, '07', 0.5173913043478261), (2.0, 3.0, 0.00390625, '08', 0.52972972972972976), (2.0, 3.0, 128.0, '09', 0.52180936995153471), (2.0, 3.0, 0.001953125, '10', 0.50653594771241828), (2.0, 3.0, 0.0009765625, '11', 0.54974271012006859), (2.0, 3.0, 0.00390625, '12', 0.53919860627177696), (2.0, 3.0, 0.0078125, '13', 0.52066115702479343), (2.0, 3.0, 0.00390625, '14', 0.54166666666666663), (2.0, 3.0, 0.0009765625, '15', 0.53239202657807305), (2.0, 3.0, 0.0009765625, '16', 0.53104026845637586), (2.0, 3.0, 0.015625, '17', 0.52173913043478259), (2.0, 3.0, 64.0, '18', 0.53150912106135983), (2.0, 3.0, 0.0009765625, '19', 0.53057851239669418), (2.0, 3.0, 0.001953125, '20', 0.53565640194489461), (2.0, 3.5, 0.015625, '01', 0.53536977491961413), (2.0, 3.5, 0.0009765625, '02', 0.5313001605136437), (2.0, 3.5, 6.103515625e-05, '03', 0.52312599681020733), (2.0, 3.5, 0.000244140625, '04', 0.52131147540983602), (2.0, 3.5, 2.0, '06', 0.52435064935064934), (2.0, 3.5, 0.001953125, '07', 0.52000000000000002), (2.0, 3.5, 0.001953125, '08', 0.53513513513513511), (2.0, 3.5, 0.0078125, '09', 0.52180936995153471), (2.0, 3.5, 3.0517578125e-05, '10', 0.50408496732026142), (2.0, 3.5, 0.001953125, '11', 0.54974271012006859), (2.0, 3.5, 0.0009765625, '12', 0.54878048780487809), (2.0, 3.5, 64.0, '13', 0.52148760330578514), (2.0, 3.5, 0.00390625, '14', 0.54761904761904767), (2.0, 3.5, 0.015625, '15', 0.54651162790697672), (2.0, 3.5, 0.001953125, '16', 0.54026845637583898), (2.0, 3.5, 0.00390625, '17', 0.52257525083612044), (2.0, 3.5, 0.00390625, '18', 0.53067993366500832), (2.0, 3.5, 0.0009765625, '19', 0.53966942148760333), (2.0, 3.5, 0.001953125, '20', 0.54213938411669371), (2.0, 4.0, 0.001953125, '01', 0.5337620578778135), (2.0, 4.0, 0.0009765625, '02', 0.5337078651685393), (2.0, 4.0, 0.0009765625, '03', 0.51674641148325362), (2.0, 4.0, 0.000244140625, '04', 0.51229508196721307), (2.0, 4.0, 64.0, '06', 0.5267857142857143), (2.0, 4.0, 0.0001220703125, '07', 0.51478260869565218), (2.0, 4.0, 0.0009765625, '08', 0.53243243243243243), (2.0, 4.0, 0.00048828125, '09', 0.52019386106623589), (2.0, 4.0, 2.384185791015625e-07, '10', 0.5106209150326797), (2.0, 4.0, 0.001953125, '11', 0.54716981132075471), (2.0, 4.0, 0.0078125, '12', 0.54703832752613235), (2.0, 4.0, 64.0, '13', 0.53305785123966942), (2.0, 4.0, 0.001953125, '14', 0.54761904761904767), (2.0, 4.0, 0.0625, '15', 0.53903654485049834), (2.0, 4.0, 0.001953125, '16', 0.54530201342281881), (2.0, 4.0, 0.00048828125, '17', 0.52173913043478259), (2.0, 4.0, 0.001953125, '18', 0.52155887230514097), (2.0, 4.0, 0.00390625, '19', 0.52975206611570247), (2.0, 4.0, 8.0, '20', 0.54051863857374394), (2.0, 4.5, 0.0009765625, '01', 0.53456591639871387), (2.0, 4.5, 0.00048828125, '02', 0.5345104333868379), (2.0, 4.5, 0.001953125, '03', 0.51993620414673047), (2.0, 4.5, 0.00048828125, '04', 0.51065573770491801), (2.0, 4.5, 64.0, '06', 0.51623376623376627), (2.0, 4.5, 0.000244140625, '07', 0.52000000000000002), (2.0, 4.5, 0.0009765625, '08', 0.53423423423423422), (2.0, 4.5, 0.00048828125, '09', 0.52827140549273022), (2.0, 4.5, 1.1920928955078125e-07, '10', 0.50490196078431371), (2.0, 4.5, 0.0009765625, '11', 0.55403087478559176), (2.0, 4.5, 0.001953125, '12', 0.54529616724738672), (2.0, 4.5, 0.03125, '13', 0.52479338842975209), (2.0, 4.5, 0.00390625, '14', 0.54591836734693877), (2.0, 4.5, 0.0009765625, '15', 0.53820598006644516), (2.0, 4.5, 0.00390625, '16', 0.54110738255033553), (2.0, 4.5, 9.3132257461547852e-10, '17', 0.52257525083612044), (2.0, 4.5, 0.00390625, '18', 0.52487562189054726), (2.0, 4.5, 0.001953125, '19', 0.53388429752066113), (2.0, 4.5, 0.25, '20', 0.53160453808752028), (2.5, 0.5, 0.00390625, '01', 0.53295819935691313), (2.5, 0.5, 0.125, '02', 0.5457463884430177), (2.5, 0.5, 1.0, '03', 0.53748006379585322), (2.5, 0.5, 8.0, '04', 0.55000000000000004), (2.5, 0.5, 1.0, '06', 0.54870129870129869), (2.5, 0.5, 0.25, '07', 0.54521739130434788), (2.5, 0.5, 0.25, '08', 0.52882882882882887), (2.5, 0.5, 0.00390625, '09', 0.53231017770597733), (2.5, 0.5, 0.5, '10', 0.55637254901960786), (2.5, 0.5, 8.0, '11', 0.56946826758147517), (2.5, 0.5, 32.0, '12', 0.56097560975609762), (2.5, 0.5, 1.0, '13', 0.53636363636363638), (2.5, 0.5, 0.0009765625, '14', 0.52636054421768708), (2.5, 0.5, 0.25, '15', 0.54069767441860461), (2.5, 0.5, 64.0, '16', 0.53942953020134232), (2.5, 0.5, 1.0, '17', 0.54933110367892979), (2.5, 0.5, 2.0, '18', 0.54477611940298509), (2.5, 0.5, 0.03125, '19', 0.5504132231404959), (2.5, 0.5, 2.0, '20', 0.53970826580226905), (2.5, 1.0, 0.5, '01', 0.5361736334405145), (2.5, 1.0, 32.0, '02', 0.53691813804173349), (2.5, 1.0, 0.0625, '03', 0.54146730462519932), (2.5, 1.0, 0.015625, '04', 0.53278688524590168), (2.5, 1.0, 32.0, '06', 0.53733766233766234), (2.5, 1.0, 0.125, '07', 0.5347826086956522), (2.5, 1.0, 0.03125, '08', 0.54234234234234235), (2.5, 1.0, 0.015625, '09', 0.54604200323101781), (2.5, 1.0, 0.03125, '10', 0.54820261437908502), (2.5, 1.0, 0.0625, '11', 0.55746140651801024), (2.5, 1.0, 0.125, '12', 0.55400696864111498), (2.5, 1.0, 0.00390625, '13', 0.54132231404958675), (2.5, 1.0, 0.03125, '14', 0.54846938775510201), (2.5, 1.0, 0.0625, '15', 0.54235880398671099), (2.5, 1.0, 0.001953125, '16', 0.5436241610738255), (2.5, 1.0, 0.0625, '17', 0.54096989966555187), (2.5, 1.0, 0.001953125, '18', 0.54809286898839138), (2.5, 1.0, 0.125, '19', 0.5446280991735537), (2.5, 1.0, 0.125, '20', 0.55186385737439225), (2.5, 1.5, 64.0, '01', 0.5313504823151125), (2.5, 1.5, 16.0, '02', 0.5345104333868379), (2.5, 1.5, 0.0078125, '03', 0.52791068580542266), (2.5, 1.5, 64.0, '04', 0.5229508196721312), (2.5, 1.5, 64.0, '06', 0.52516233766233766), (2.5, 1.5, 16.0, '07', 0.53565217391304343), (2.5, 1.5, 0.25, '08', 0.52792792792792798), (2.5, 1.5, 0.001953125, '09', 0.53231017770597733), (2.5, 1.5, 0.00390625, '10', 0.53186274509803921), (2.5, 1.5, 32.0, '11', 0.5480274442538593), (2.5, 1.5, 64.0, '12', 0.55487804878048785), (2.5, 1.5, 0.03125, '13', 0.54628099173553724), (2.5, 1.5, 0.03125, '14', 0.54251700680272108), (2.5, 1.5, 64.0, '15', 0.53239202657807305), (2.5, 1.5, 0.0078125, '16', 0.53691275167785235), (2.5, 1.5, 0.5, '17', 0.52675585284280935), (2.5, 1.5, 0.001953125, '18', 0.52902155887230518), (2.5, 1.5, 0.0078125, '19', 0.52561983471074381), (2.5, 1.5, 2.0, '20', 0.52917341977309562), (2.5, 2.0, 0.0078125, '01', 0.53697749196141475), (2.5, 2.0, 0.0625, '02', 0.5353130016051364), (2.5, 2.0, 0.0078125, '03', 0.53827751196172247), (2.5, 2.0, 0.00390625, '04', 0.52950819672131144), (2.5, 2.0, 32.0, '06', 0.52840909090909094), (2.5, 2.0, 0.0078125, '07', 0.54260869565217396), (2.5, 2.0, 2.0, '08', 0.52882882882882887), (2.5, 2.0, 0.03125, '09', 0.53150242326332797), (2.5, 2.0, 16.0, '10', 0.53349673202614378), (2.5, 2.0, 32.0, '11', 0.55831903945111494), (2.5, 2.0, 32.0, '12', 0.56881533101045301), (2.5, 2.0, 0.03125, '13', 0.52809917355371905), (2.5, 2.0, 0.03125, '14', 0.53656462585034015), (2.5, 2.0, 0.00390625, '15', 0.53405315614617943), (2.5, 2.0, 0.5, '16', 0.52936241610738255), (2.5, 2.0, 32.0, '17', 0.52591973244147161), (2.5, 2.0, 0.03125, '18', 0.54063018242122718), (2.5, 2.0, 32.0, '19', 0.53388429752066113), (2.5, 2.0, 0.015625, '20', 0.54132901134521882), (2.5, 2.5, 0.015625, '01', 0.52572347266881025), (2.5, 2.5, 4.0, '02', 0.5465489566613162), (2.5, 2.5, 0.0078125, '03', 0.52551834130781494), (2.5, 2.5, 0.0078125, '04', 0.53524590163934427), (2.5, 2.5, 0.00390625, '06', 0.5316558441558441), (2.5, 2.5, 0.125, '07', 0.53826086956521735), (2.5, 2.5, 64.0, '08', 0.53963963963963968), (2.5, 2.5, 0.001953125, '09', 0.54119547657512113), (2.5, 2.5, 16.0, '10', 0.54003267973856206), (2.5, 2.5, 0.03125, '11', 0.55917667238421953), (2.5, 2.5, 16.0, '12', 0.55923344947735187), (2.5, 2.5, 0.0078125, '13', 0.52396694214876038), (2.5, 2.5, 0.001953125, '14', 0.53486394557823125), (2.5, 2.5, 0.001953125, '15', 0.5357142857142857), (2.5, 2.5, 0.015625, '16', 0.53355704697986572), (2.5, 2.5, 0.0009765625, '17', 0.52257525083612044), (2.5, 2.5, 0.0625, '18', 0.54643449419568824), (2.5, 2.5, 64.0, '19', 0.53388429752066113), (2.5, 2.5, 0.015625, '20', 0.54213938411669371), (2.5, 3.0, 64.0, '01', 0.54581993569131837), (2.5, 3.0, 2.0, '02', 0.5457463884430177), (2.5, 3.0, 0.0625, '03', 0.52551834130781494), (2.5, 3.0, 0.00390625, '04', 0.52459016393442626), (2.5, 3.0, 0.03125, '06', 0.53327922077922074), (2.5, 3.0, 32.0, '07', 0.53565217391304343), (2.5, 3.0, 4.0, '08', 0.53873873873873879), (2.5, 3.0, 0.0625, '09', 0.53150242326332797), (2.5, 3.0, 64.0, '10', 0.54166666666666663), (2.5, 3.0, 8.0, '11', 0.565180102915952), (2.5, 3.0, 16.0, '12', 0.57229965156794427), (2.5, 3.0, 0.015625, '13', 0.52727272727272723), (2.5, 3.0, 0.001953125, '14', 0.54506802721088432), (2.5, 3.0, 8.0, '15', 0.54235880398671099), (2.5, 3.0, 0.015625, '16', 0.53104026845637586), (2.5, 3.0, 0.00390625, '17', 0.52424749163879603), (2.5, 3.0, 0.03125, '18', 0.54394693200663347), (2.5, 3.0, 32.0, '19', 0.53553719008264467), (2.5, 3.0, 64.0, '20', 0.55105348460291737), (2.5, 3.5, 0.001953125, '01', 0.52974276527331188), (2.5, 3.5, 0.0625, '02', 0.5417335473515249), (2.5, 3.5, 16.0, '03', 0.52551834130781494), (2.5, 3.5, 0.0625, '04', 0.52786885245901638), (2.5, 3.5, 2.0, '06', 0.51623376623376627), (2.5, 3.5, 16.0, '07', 0.53739130434782612), (2.5, 3.5, 0.00390625, '08', 0.54414414414414414), (2.5, 3.5, 0.015625, '09', 0.54281098546042006), (2.5, 3.5, 32.0, '10', 0.54248366013071891), (2.5, 3.5, 64.0, '11', 0.55574614065180106), (2.5, 3.5, 32.0, '12', 0.55836236933797911), (2.5, 3.5, 0.015625, '13', 0.53305785123966942), (2.5, 3.5, 0.03125, '14', 0.55102040816326525), (2.5, 3.5, 0.0078125, '15', 0.53488372093023251), (2.5, 3.5, 0.015625, '16', 0.54194630872483218), (2.5, 3.5, 0.0078125, '17', 0.53010033444816052), (2.5, 3.5, 0.00390625, '18', 0.54726368159203975), (2.5, 3.5, 0.00390625, '19', 0.5380165289256198), (2.5, 3.5, 0.001953125, '20', 0.54943273905996759), (2.5, 4.0, 8.0, '01', 0.54019292604501612), (2.5, 4.0, 4.0, '02', 0.5449438202247191), (2.5, 4.0, 4.0, '03', 0.52950558213716103), (2.5, 4.0, 0.0078125, '04', 0.52540983606557379), (2.5, 4.0, 64.0, '06', 0.53409090909090906), (2.5, 4.0, 0.00390625, '07', 0.54000000000000004), (2.5, 4.0, 0.001953125, '08', 0.5495495495495496), (2.5, 4.0, 0.015625, '09', 0.54442649434571888), (2.5, 4.0, 16.0, '10', 0.54656862745098034), (2.5, 4.0, 16.0, '11', 0.55317324185248717), (2.5, 4.0, 32.0, '12', 0.5714285714285714), (2.5, 4.0, 16.0, '13', 0.531404958677686), (2.5, 4.0, 0.0078125, '14', 0.55442176870748294), (2.5, 4.0, 16.0, '15', 0.54900332225913617), (2.5, 4.0, 0.015625, '16', 0.54614093959731547), (2.5, 4.0, 0.0078125, '17', 0.53428093645484953), (2.5, 4.0, 0.0625, '18', 0.55140961857379767), (2.5, 4.0, 8.0, '19', 0.54793388429752066), (2.5, 4.0, 8.0, '20', 0.54943273905996759), (2.5, 4.5, 0.25, '01', 0.53456591639871387), (2.5, 4.5, 16.0, '02', 0.5385232744783307), (2.5, 4.5, 64.0, '03', 0.53508771929824561), (2.5, 4.5, 0.0078125, '04', 0.52459016393442626), (2.5, 4.5, 2.0, '06', 0.53652597402597402), (2.5, 4.5, 0.00390625, '07', 0.53826086956521735), (2.5, 4.5, 0.00390625, '08', 0.5495495495495496), (2.5, 4.5, 0.03125, '09', 0.54119547657512113), (2.5, 4.5, 64.0, '10', 0.54003267973856206), (2.5, 4.5, 64.0, '11', 0.55231560891938247), (2.5, 4.5, 32.0, '12', 0.56620209059233451), (2.5, 4.5, 64.0, '13', 0.53884297520661162), (2.5, 4.5, 0.0078125, '14', 0.5535714285714286), (2.5, 4.5, 64.0, '15', 0.54318936877076407), (2.5, 4.5, 32.0, '16', 0.54194630872483218), (2.5, 4.5, 0.00390625, '17', 0.54096989966555187), (2.5, 4.5, 0.03125, '18', 0.54726368159203975), (2.5, 4.5, 32.0, '19', 0.54132231404958675), (2.5, 4.5, 8.0, '20', 0.54619124797406804), (3.0, 0.5, 8.0, '01', 0.53858520900321538), (3.0, 0.5, 2.0, '02', 0.5385232744783307), (3.0, 0.5, 0.001953125, '03', 0.53668261562998409), (3.0, 0.5, 1.0, '04', 0.53196721311475414), (3.0, 0.5, 0.001953125, '06', 0.53409090909090906), (3.0, 0.5, 0.015625, '07', 0.5304347826086957), (3.0, 0.5, 128.0, '08', 0.50180180180180178), (3.0, 0.5, 0.03125, '09', 0.53392568659127626), (3.0, 0.5, 16.0, '10', 0.52696078431372551), (3.0, 0.5, 0.00390625, '11', 0.53430531732418529), (3.0, 0.5, 0.125, '12', 0.5383275261324042), (3.0, 0.5, 0.03125, '13', 0.5380165289256198), (3.0, 0.5, 0.03125, '14', 0.54251700680272108), (3.0, 0.5, 0.5, '15', 0.53820598006644516), (3.0, 0.5, 0.125, '16', 0.55369127516778527), (3.0, 0.5, 0.0078125, '17', 0.53678929765886285), (3.0, 0.5, 0.0078125, '18', 0.53316749585406298), (3.0, 0.5, 0.25, '19', 0.54958677685950408), (3.0, 0.5, 0.015625, '20', 0.54457050243111826), (3.0, 1.0, 0.015625, '01', 0.54421221864951763), (3.0, 1.0, 1.0, '02', 0.5425361155698234), (3.0, 1.0, 4.0, '03', 0.55901116427432218), (3.0, 1.0, 0.125, '04', 0.55491803278688523), (3.0, 1.0, 0.125, '06', 0.56899350649350644), (3.0, 1.0, 0.5, '07', 0.55391304347826087), (3.0, 1.0, 1.0, '08', 0.54684684684684681), (3.0, 1.0, 0.125, '09', 0.55250403877221321), (3.0, 1.0, 0.03125, '10', 0.56290849673202614), (3.0, 1.0, 0.25, '11', 0.55660377358490565), (3.0, 1.0, 64.0, '12', 0.56010452961672474), (3.0, 1.0, 0.5, '13', 0.55950413223140494), (3.0, 1.0, 8.0, '14', 0.56207482993197277), (3.0, 1.0, 64.0, '15', 0.54401993355481726), (3.0, 1.0, 0.0625, '16', 0.55788590604026844), (3.0, 1.0, 0.001953125, '17', 0.55769230769230771), (3.0, 1.0, 1.0, '18', 0.5713101160862355), (3.0, 1.0, 0.03125, '19', 0.55206611570247932), (3.0, 1.0, 32.0, '20', 0.55267423014586714), (3.0, 1.5, 64.0, '01', 0.5361736334405145), (3.0, 1.5, 0.015625, '02', 0.53691813804173349), (3.0, 1.5, 0.125, '03', 0.54625199362041466), (3.0, 1.5, 4.0, '04', 0.54262295081967216), (3.0, 1.5, 4.0, '06', 0.55032467532467533), (3.0, 1.5, 64.0, '07', 0.52869565217391301), (3.0, 1.5, 0.0009765625, '08', 0.51531531531531527), (3.0, 1.5, 1.0, '09', 0.52907915993537968), (3.0, 1.5, 0.03125, '10', 0.53186274509803921), (3.0, 1.5, 0.125, '11', 0.55574614065180106), (3.0, 1.5, 16.0, '12', 0.54616724738675959), (3.0, 1.5, 8.0, '13', 0.54958677685950408), (3.0, 1.5, 16.0, '14', 0.54421768707482998), (3.0, 1.5, 4.0, '15', 0.53986710963455153), (3.0, 1.5, 16.0, '16', 0.55704697986577179), (3.0, 1.5, 0.125, '17', 0.5418060200668896), (3.0, 1.5, 0.5, '18', 0.54063018242122718), (3.0, 1.5, 1.0, '19', 0.53719008264462809), (3.0, 1.5, 64.0, '20', 0.53241491085899517), (3.0, 2.0, 32.0, '01', 0.52090032154340837), (3.0, 2.0, 32.0, '02', 0.5168539325842697), (3.0, 2.0, 128.0, '03', 0.51754385964912286), (3.0, 2.0, 0.0625, '04', 0.51967213114754096), (3.0, 2.0, 64.0, '06', 0.52840909090909094), (3.0, 2.0, 0.00390625, '07', 0.50347826086956526), (3.0, 2.0, 0.0625, '08', 0.52072072072072073), (3.0, 2.0, 128.0, '09', 0.51534733441033931), (3.0, 2.0, 128.0, '10', 0.5106209150326797), (3.0, 2.0, 0.125, '11', 0.53945111492281306), (3.0, 2.0, 8.0, '12', 0.52961672473867594), (3.0, 2.0, 32.0, '13', 0.53636363636363638), (3.0, 2.0, 0.015625, '14', 0.52891156462585032), (3.0, 2.0, 64.0, '15', 0.52076411960132896), (3.0, 2.0, 0.0078125, '16', 0.52013422818791943), (3.0, 2.0, 0.00390625, '17', 0.51421404682274252), (3.0, 2.0, 0.0078125, '18', 0.51658374792703154), (3.0, 2.0, 0.0625, '19', 0.50909090909090904), (3.0, 2.0, 64.0, '20', 0.52593192868719607), (3.0, 2.5, 64.0, '01', 0.52090032154340837), (3.0, 2.5, 128.0, '02', 0.5192616372391653), (3.0, 2.5, 128.0, '03', 0.51355661881977677), (3.0, 2.5, 32.0, '04', 0.50737704918032789), (3.0, 2.5, 32.0, '06', 0.50649350649350644), (3.0, 2.5, 64.0, '07', 0.52086956521739125), (3.0, 2.5, 0.015625, '08', 0.53063063063063065), (3.0, 2.5, 1.0, '09', 0.52827140549273022), (3.0, 2.5, 256.0, '10', 0.50490196078431371), (3.0, 2.5, 0.00390625, '11', 0.54116638078902235), (3.0, 2.5, 8.0, '12', 0.53397212543554007), (3.0, 2.5, 0.125, '13', 0.52314049586776856), (3.0, 2.5, 0.0625, '14', 0.53486394557823125), (3.0, 2.5, 16.0, '15', 0.51495016611295685), (3.0, 2.5, 0.03125, '16', 0.5226510067114094), (3.0, 2.5, 0.0078125, '17', 0.52006688963210701), (3.0, 2.5, 0.015625, '18', 0.52819237147595355), (3.0, 2.5, 64.0, '19', 0.5190082644628099), (3.0, 2.5, 128.0, '20', 0.52188006482982174), (3.0, 3.0, 4.0, '01', 0.5313504823151125), (3.0, 3.0, 16.0, '02', 0.5232744783306581), (3.0, 3.0, 0.5, '03', 0.5103668261562998), (3.0, 3.0, 0.5, '04', 0.51721311475409837), (3.0, 3.0, 0.0009765625, '06', 0.50405844155844159), (3.0, 3.0, 0.00390625, '07', 0.50782608695652176), (3.0, 3.0, 0.0078125, '08', 0.53693693693693689), (3.0, 3.0, 0.015625, '09', 0.51453957996768984), (3.0, 3.0, 0.00048828125, '10', 0.51143790849673199), (3.0, 3.0, 0.001953125, '11', 0.54545454545454541), (3.0, 3.0, 32.0, '12', 0.55662020905923348), (3.0, 3.0, 0.015625, '13', 0.51818181818181819), (3.0, 3.0, 0.25, '14', 0.55017006802721091), (3.0, 3.0, 0.001953125, '15', 0.52325581395348841), (3.0, 3.0, 0.001953125, '16', 0.53607382550335569), (3.0, 3.0, 0.00390625, '17', 0.53010033444816052), (3.0, 3.0, 0.0009765625, '18', 0.53316749585406298), (3.0, 3.0, 0.001953125, '19', 0.52809917355371905), (3.0, 3.0, 64.0, '20', 0.52269043760129663), (3.0, 3.5, 0.0009765625, '01', 0.51768488745980712), (3.0, 3.5, 64.0, '02', 0.5240770465489567), (3.0, 3.5, 0.001953125, '03', 0.51515151515151514), (3.0, 3.5, 32.0, '04', 0.50983606557377048), (3.0, 3.5, 128.0, '06', 0.50405844155844159), (3.0, 3.5, 256.0, '07', 0.50695652173913042), (3.0, 3.5, 32.0, '08', 0.54324324324324325), (3.0, 3.5, 0.0009765625, '09', 0.51615508885298866), (3.0, 3.5, 256.0, '10', 0.50653594771241828), (3.0, 3.5, 0.001953125, '11', 0.54631217838765012), (3.0, 3.5, 8.0, '12', 0.54268292682926833), (3.0, 3.5, 0.0078125, '13', 0.51818181818181819), (3.0, 3.5, 1.0, '14', 0.53911564625850339), (3.0, 3.5, 32.0, '15', 0.51495016611295685), (3.0, 3.5, 0.001953125, '16', 0.52936241610738255), (3.0, 3.5, 0.03125, '17', 0.51672240802675584), (3.0, 3.5, 0.0009765625, '18', 0.53482587064676612), (3.0, 3.5, 0.001953125, '19', 0.52644628099173552), (3.0, 3.5, 0.0009765625, '20', 0.52836304700162073), (3.0, 4.0, 64.0, '01', 0.52491961414791), (3.0, 4.0, 0.0009765625, '02', 0.5168539325842697), (3.0, 4.0, 0.001953125, '03', 0.51515151515151514), (3.0, 4.0, 0.0001220703125, '04', 0.50819672131147542), (3.0, 4.0, 9.3132257461547852e-10, '06', 0.50324675324675328), (3.0, 4.0, 0.00390625, '07', 0.50695652173913042), (3.0, 4.0, 64.0, '08', 0.54234234234234235), (3.0, 4.0, 0.0009765625, '09', 0.51938610662358642), (3.0, 4.0, 6.103515625e-05, '10', 0.5106209150326797), (3.0, 4.0, 0.0009765625, '11', 0.54116638078902235), (3.0, 4.0, 64.0, '12', 0.54703832752613235), (3.0, 4.0, 128.0, '13', 0.51818181818181819), (3.0, 4.0, 2.0, '14', 0.54336734693877553), (3.0, 4.0, 0.001953125, '15', 0.51328903654485047), (3.0, 4.0, 0.00390625, '16', 0.52768456375838924), (3.0, 4.0, 64.0, '17', 0.51337792642140467), (3.0, 4.0, 0.0009765625, '18', 0.5265339966832504), (3.0, 4.0, 0.001953125, '19', 0.51487603305785123), (3.0, 4.0, 0.001953125, '20', 0.52512155591572118), (3.0, 4.5, 32.0, '01', 0.52491961414791), (3.0, 4.5, 16.0, '02', 0.5280898876404494), (3.0, 4.5, 64.0, '03', 0.51674641148325362), (3.0, 4.5, 32.0, '04', 0.51065573770491801), (3.0, 4.5, 0.00048828125, '06', 0.50811688311688308), (3.0, 4.5, 0.001953125, '07', 0.51391304347826083), (3.0, 4.5, 0.015625, '08', 0.5477477477477477), (3.0, 4.5, 8.0, '09', 0.52261712439418417), (3.0, 4.5, 1.9073486328125e-06, '10', 0.50653594771241828), (3.0, 4.5, 0.0009765625, '11', 0.54631217838765012), (3.0, 4.5, 32.0, '12', 0.54616724738675959), (3.0, 4.5, 64.0, '13', 0.52644628099173552), (3.0, 4.5, 16.0, '14', 0.54421768707482998), (3.0, 4.5, 128.0, '15', 0.51079734219269102), (3.0, 4.5, 0.00390625, '16', 0.52852348993288589), (3.0, 4.5, 0.0078125, '17', 0.51170568561872909), (3.0, 4.5, 0.0009765625, '18', 0.53067993366500832), (3.0, 4.5, 0.001953125, '19', 0.51322314049586781), (3.0, 4.5, 0.001953125, '20', 0.52350081037277152), (3.5, 0.5, 0.0078125, '01', 0.51045016077170413), (3.5, 0.5, 0.0078125, '02', 0.5136436597110754), (3.5, 0.5, 1024.0, '03', 0.50159489633173848), (3.5, 0.5, 0.00390625, '04', 0.5229508196721312), (3.5, 0.5, 3.0517578125e-05, '06', 0.53003246753246758), (3.5, 0.5, 8.0, '07', 0.52260869565217394), (3.5, 0.5, 0.001953125, '08', 0.51081081081081081), (3.5, 0.5, 0.0009765625, '09', 0.50403877221324722), (3.5, 0.5, 0.00390625, '10', 0.50245098039215685), (3.5, 0.5, 0.0009765625, '11', 0.52229845626072036), (3.5, 0.5, 0.000244140625, '12', 0.50348432055749126), (3.5, 0.5, 0.00048828125, '13', 0.52314049586776856), (3.5, 0.5, 7.62939453125e-06, '14', 0.50085034013605445), (3.5, 0.5, 0.0009765625, '15', 0.50996677740863783), (3.5, 0.5, 0.001953125, '16', 0.5151006711409396), (3.5, 0.5, 6.103515625e-05, '17', 0.50501672240802675), (3.5, 0.5, 0.001953125, '18', 0.52736318407960203), (3.5, 0.5, 0.0078125, '19', 0.51157024793388428), (3.5, 0.5, 0.0078125, '20', 0.51458670988654787), (3.5, 1.0, 0.00390625, '01', 0.51929260450160775), (3.5, 1.0, 0.015625, '02', 0.5393258426966292), (3.5, 1.0, 0.00390625, '03', 0.53508771929824561), (3.5, 1.0, 0.125, '04', 0.53360655737704921), (3.5, 1.0, 0.00390625, '06', 0.5267857142857143), (3.5, 1.0, 0.0625, '07', 0.54521739130434788), (3.5, 1.0, 0.03125, '08', 0.50990990990990992), (3.5, 1.0, 2.0, '09', 0.53392568659127626), (3.5, 1.0, 1.0, '10', 0.52042483660130723), (3.5, 1.0, 0.0078125, '11', 0.54545454545454541), (3.5, 1.0, 0.0078125, '12', 0.54529616724738672), (3.5, 1.0, 0.0625, '13', 0.53305785123966942), (3.5, 1.0, 0.0078125, '14', 0.53911564625850339), (3.5, 1.0, 0.125, '15', 0.54069767441860461), (3.5, 1.0, 0.015625, '16', 0.54697986577181212), (3.5, 1.0, 0.0078125, '17', 0.52759197324414719), (3.5, 1.0, 0.001953125, '18', 0.54643449419568824), (3.5, 1.0, 0.0625, '19', 0.53471074380165284), (3.5, 1.0, 0.00390625, '20', 0.53241491085899517), (3.5, 1.5, 0.015625, '01', 0.52170418006430863), (3.5, 1.5, 0.0078125, '02', 0.528892455858748), (3.5, 1.5, 64.0, '03', 0.53030303030303028), (3.5, 1.5, 0.00390625, '04', 0.53442622950819674), (3.5, 1.5, 1.0, '06', 0.54139610389610393), (3.5, 1.5, 2.0, '07', 0.54608695652173911), (3.5, 1.5, 0.015625, '08', 0.53243243243243243), (3.5, 1.5, 2.0, '09', 0.52746365105008075), (3.5, 1.5, 2.0, '10', 0.53594771241830064), (3.5, 1.5, 0.0625, '11', 0.55403087478559176), (3.5, 1.5, 0.0078125, '12', 0.55052264808362372), (3.5, 1.5, 32.0, '13', 0.53636363636363638), (3.5, 1.5, 0.125, '14', 0.52806122448979587), (3.5, 1.5, 0.5, '15', 0.52657807308970095), (3.5, 1.5, 0.03125, '16', 0.54446308724832215), (3.5, 1.5, 8.0, '17', 0.5317725752508361), (3.5, 1.5, 0.0078125, '18', 0.53897180762852404), (3.5, 1.5, 0.00390625, '19', 0.53305785123966942), (3.5, 1.5, 0.0625, '20', 0.55105348460291737), (3.5, 2.0, 0.00390625, '01', 0.5313504823151125), (3.5, 2.0, 0.0078125, '02', 0.5329052969502408), (3.5, 2.0, 0.0625, '03', 0.5287081339712919), (3.5, 2.0, 0.125, '04', 0.52786885245901638), (3.5, 2.0, 0.25, '06', 0.52516233766233766), (3.5, 2.0, 0.03125, '07', 0.5304347826086957), (3.5, 2.0, 0.015625, '08', 0.53513513513513511), (3.5, 2.0, 32.0, '09', 0.52988691437802904), (3.5, 2.0, 0.5, '10', 0.53594771241830064), (3.5, 2.0, 0.00390625, '11', 0.55831903945111494), (3.5, 2.0, 1.0, '12', 0.54703832752613235), (3.5, 2.0, 0.0078125, '13', 0.53305785123966942), (3.5, 2.0, 0.015625, '14', 0.54251700680272108), (3.5, 2.0, 0.03125, '15', 0.53156146179401997), (3.5, 2.0, 0.0078125, '16', 0.53942953020134232), (3.5, 2.0, 0.0009765625, '17', 0.51672240802675584), (3.5, 2.0, 0.001953125, '18', 0.52736318407960203), (3.5, 2.0, 0.00390625, '19', 0.5380165289256198), (3.5, 2.0, 0.015625, '20', 0.54781199351701781), (3.5, 2.5, 0.0078125, '01', 0.51929260450160775), (3.5, 2.5, 0.015625, '02', 0.52969502407704649), (3.5, 2.5, 64.0, '03', 0.51196172248803828), (3.5, 2.5, 0.0078125, '04', 0.52049180327868849), (3.5, 2.5, 0.00048828125, '06', 0.51461038961038963), (3.5, 2.5, 0.015625, '07', 0.54434782608695653), (3.5, 2.5, 0.0625, '08', 0.54414414414414414), (3.5, 2.5, 0.0078125, '09', 0.52342487883683364), (3.5, 2.5, 0.0078125, '10', 0.52287581699346408), (3.5, 2.5, 0.015625, '11', 0.54373927958833623), (3.5, 2.5, 0.03125, '12', 0.54529616724738672), (3.5, 2.5, 0.0625, '13', 0.53223140495867771), (3.5, 2.5, 0.03125, '14', 0.53826530612244894), (3.5, 2.5, 0.015625, '15', 0.52823920265780733), (3.5, 2.5, 0.015625, '16', 0.53859060402684567), (3.5, 2.5, 64.0, '17', 0.52341137123745818), (3.5, 2.5, 0.0625, '18', 0.52155887230514097), (3.5, 2.5, 0.0078125, '19', 0.53057851239669418), (3.5, 2.5, 0.00390625, '20', 0.52917341977309562), (3.5, 3.0, 0.00390625, '01', 0.52411575562700963), (3.5, 3.0, 64.0, '02', 0.5345104333868379), (3.5, 3.0, 0.015625, '03', 0.51834130781499199), (3.5, 3.0, 0.0078125, '04', 0.53852459016393439), (3.5, 3.0, 0.00048828125, '06', 0.52029220779220775), (3.5, 3.0, 2.0, '07', 0.53391304347826085), (3.5, 3.0, 0.5, '08', 0.53063063063063065), (3.5, 3.0, 64.0, '09', 0.53311793214862679), (3.5, 3.0, 0.015625, '10', 0.51960784313725494), (3.5, 3.0, 0.00390625, '11', 0.55403087478559176), (3.5, 3.0, 0.0078125, '12', 0.54442508710801396), (3.5, 3.0, 0.5, '13', 0.531404958677686), (3.5, 3.0, 0.0625, '14', 0.54931972789115646), (3.5, 3.0, 0.0078125, '15', 0.52574750830564787), (3.5, 3.0, 0.015625, '16', 0.53942953020134232), (3.5, 3.0, 32.0, '17', 0.52508361204013376), (3.5, 3.0, 0.00390625, '18', 0.52404643449419563), (3.5, 3.0, 0.0625, '19', 0.52975206611570247), (3.5, 3.0, 16.0, '20', 0.53970826580226905), (3.5, 3.5, 0.0078125, '01', 0.52491961414791), (3.5, 3.5, 0.03125, '02', 0.5353130016051364), (3.5, 3.5, 0.0625, '03', 0.52073365231259972), (3.5, 3.5, 0.125, '04', 0.53442622950819674), (3.5, 3.5, 0.00048828125, '06', 0.51704545454545459), (3.5, 3.5, 0.5, '07', 0.53826086956521735), (3.5, 3.5, 0.00390625, '08', 0.53333333333333333), (3.5, 3.5, 8.0, '09', 0.53311793214862679), (3.5, 3.5, 0.0078125, '10', 0.52696078431372551), (3.5, 3.5, 0.00390625, '11', 0.55831903945111494), (3.5, 3.5, 64.0, '12', 0.54703832752613235), (3.5, 3.5, 64.0, '13', 0.53471074380165284), (3.5, 3.5, 0.0078125, '14', 0.54336734693877553), (3.5, 3.5, 128.0, '15', 0.52657807308970095), (3.5, 3.5, 0.03125, '16', 0.53859060402684567), (3.5, 3.5, 0.0078125, '17', 0.52257525083612044), (3.5, 3.5, 0.0078125, '18', 0.53233830845771146), (3.5, 3.5, 0.0078125, '19', 0.53966942148760333), (3.5, 3.5, 0.0078125, '20', 0.53160453808752028), (3.5, 4.0, 0.00390625, '01', 0.52572347266881025), (3.5, 4.0, 0.015625, '02', 0.5337078651685393), (3.5, 4.0, 8.0, '03', 0.51834130781499199), (3.5, 4.0, 0.015625, '04', 0.52540983606557379), (3.5, 4.0, 0.00048828125, '06', 0.51623376623376627), (3.5, 4.0, 0.03125, '07', 0.54000000000000004), (3.5, 4.0, 0.015625, '08', 0.53423423423423422), (3.5, 4.0, 0.00048828125, '09', 0.53150242326332797), (3.5, 4.0, 0.00390625, '10', 0.53676470588235292), (3.5, 4.0, 0.0078125, '11', 0.56174957118353341), (3.5, 4.0, 4.0, '12', 0.54442508710801396), (3.5, 4.0, 0.015625, '13', 0.52975206611570247), (3.5, 4.0, 0.5, '14', 0.54591836734693877), (3.5, 4.0, 0.00048828125, '15', 0.52408637873754149), (3.5, 4.0, 0.0009765625, '16', 0.53942953020134232), (3.5, 4.0, 0.0625, '17', 0.52591973244147161), (3.5, 4.0, 0.0078125, '18', 0.53399668325041461), (3.5, 4.0, 0.0078125, '19', 0.53966942148760333), (3.5, 4.0, 16.0, '20', 0.53970826580226905), (3.5, 4.5, 0.00048828125, '01', 0.52572347266881025), (3.5, 4.5, 0.015625, '02', 0.5345104333868379), (3.5, 4.5, 0.0009765625, '03', 0.51594896331738438), (3.5, 4.5, 0.0009765625, '04', 0.51639344262295084), (3.5, 4.5, 0.00048828125, '06', 0.51542207792207795), (3.5, 4.5, 0.03125, '07', 0.53217391304347827), (3.5, 4.5, 16.0, '08', 0.53783783783783778), (3.5, 4.5, 128.0, '09', 0.52988691437802904), (3.5, 4.5, 0.0078125, '10', 0.52777777777777779), (3.5, 4.5, 0.0078125, '11', 0.55145797598627788), (3.5, 4.5, 16.0, '12', 0.54006968641114983), (3.5, 4.5, 0.0078125, '13', 0.52561983471074381), (3.5, 4.5, 32.0, '14', 0.54676870748299322), (3.5, 4.5, 0.00048828125, '15', 0.52740863787375414), (3.5, 4.5, 8.0, '16', 0.53691275167785235), (3.5, 4.5, 0.03125, '17', 0.52424749163879603), (3.5, 4.5, 0.0009765625, '18', 0.52819237147595355), (3.5, 4.5, 0.0078125, '19', 0.52644628099173552), (3.5, 4.5, 16.0, '20', 0.53241491085899517), (4.0, 0.5, 0.0625, '01', 0.55948553054662375), (4.0, 0.5, 0.0078125, '02', 0.557784911717496), (4.0, 0.5, 0.015625, '03', 0.54625199362041466), (4.0, 0.5, 0.001953125, '04', 0.55573770491803276), (4.0, 0.5, 0.25, '06', 0.55925324675324672), (4.0, 0.5, 0.0078125, '07', 0.53652173913043477), (4.0, 0.5, 0.0078125, '08', 0.52702702702702697), (4.0, 0.5, 0.0009765625, '09', 0.54119547657512113), (4.0, 0.5, 0.00390625, '10', 0.52777777777777779), (4.0, 0.5, 0.0078125, '11', 0.55317324185248717), (4.0, 0.5, 0.00390625, '12', 0.53745644599303133), (4.0, 0.5, 0.0078125, '13', 0.52975206611570247), (4.0, 0.5, 0.015625, '14', 0.54166666666666663), (4.0, 0.5, 0.00390625, '15', 0.53737541528239208), (4.0, 0.5, 0.015625, '16', 0.54697986577181212), (4.0, 0.5, 0.0078125, '17', 0.54598662207357862), (4.0, 0.5, 0.125, '18', 0.54643449419568824), (4.0, 0.5, 0.015625, '19', 0.56198347107438018), (4.0, 0.5, 0.00390625, '20', 0.55915721231766613), (4.0, 1.0, 0.015625, '01', 0.53536977491961413), (4.0, 1.0, 0.125, '02', 0.5409309791332263), (4.0, 1.0, 0.0009765625, '03', 0.52472089314194581), (4.0, 1.0, 0.00048828125, '04', 0.53770491803278686), (4.0, 1.0, 0.00390625, '06', 0.52029220779220775), (4.0, 1.0, 64.0, '07', 0.53217391304347827), (4.0, 1.0, 16.0, '08', 0.52342342342342341), (4.0, 1.0, 0.03125, '09', 0.53392568659127626), (4.0, 1.0, 0.00048828125, '10', 0.53921568627450978), (4.0, 1.0, 32.0, '11', 0.55060034305317329), (4.0, 1.0, 2.0, '12', 0.53135888501742157), (4.0, 1.0, 0.015625, '13', 0.51487603305785123), (4.0, 1.0, 6.103515625e-05, '14', 0.52891156462585032), (4.0, 1.0, 64.0, '15', 0.5299003322259136), (4.0, 1.0, 1.0, '16', 0.52600671140939592), (4.0, 1.0, 0.5, '17', 0.53678929765886285), (4.0, 1.0, 4.0, '18', 0.53731343283582089), (4.0, 1.0, 0.015625, '19', 0.52148760330578514), (4.0, 1.0, 64.0, '20', 0.53808752025931927), (4.0, 1.5, 0.001953125, '01', 0.53215434083601287), (4.0, 1.5, 0.0625, '02', 0.5609951845906902), (4.0, 1.5, 0.03125, '03', 0.5287081339712919), (4.0, 1.5, 0.0078125, '04', 0.53934426229508192), (4.0, 1.5, 64.0, '06', 0.53733766233766234), (4.0, 1.5, 0.0078125, '07', 0.53304347826086962), (4.0, 1.5, 8.0, '08', 0.52702702702702697), (4.0, 1.5, 0.03125, '09', 0.55008077544426492), (4.0, 1.5, 2.0, '10', 0.54738562091503273), (4.0, 1.5, 0.001953125, '11', 0.57375643224699824), (4.0, 1.5, 0.0009765625, '12', 0.54181184668989546), (4.0, 1.5, 0.001953125, '13', 0.51404958677685952), (4.0, 1.5, 0.015625, '14', 0.54251700680272108), (4.0, 1.5, 0.001953125, '15', 0.53903654485049834), (4.0, 1.5, 0.00390625, '16', 0.54949664429530198), (4.0, 1.5, 0.001953125, '17', 0.52090301003344486), (4.0, 1.5, 0.00390625, '18', 0.55721393034825872), (4.0, 1.5, 0.00390625, '19', 0.52644628099173552), (4.0, 1.5, 64.0, '20', 0.54132901134521882), (4.0, 2.0, 16.0, '01', 0.51848874598070738), (4.0, 2.0, 0.03125, '02', 0.5393258426966292), (4.0, 2.0, 0.001953125, '03', 0.52232854864433809), (4.0, 2.0, 0.0078125, '04', 0.54180327868852463), (4.0, 2.0, 4.0, '06', 0.52516233766233766), (4.0, 2.0, 0.0009765625, '07', 0.53913043478260869), (4.0, 2.0, 0.000244140625, '08', 0.51351351351351349), (4.0, 2.0, 0.0078125, '09', 0.54200323101777059), (4.0, 2.0, 0.0078125, '10', 0.53349673202614378), (4.0, 2.0, 0.0009765625, '11', 0.55317324185248717), (4.0, 2.0, 0.0078125, '12', 0.5331010452961672), (4.0, 2.0, 64.0, '13', 0.52148760330578514), (4.0, 2.0, 0.0078125, '14', 0.54081632653061229), (4.0, 2.0, 0.125, '15', 0.53239202657807305), (4.0, 2.0, 0.0009765625, '16', 0.53942953020134232), (4.0, 2.0, 0.0009765625, '17', 0.50752508361204018), (4.0, 2.0, 0.00390625, '18', 0.55555555555555558), (4.0, 2.0, 0.0078125, '19', 0.52727272727272723), (4.0, 2.0, 0.001953125, '20', 0.5307941653160454), (4.0, 2.5, 0.0009765625, '01', 0.51929260450160775), (4.0, 2.5, 0.015625, '02', 0.5313001605136437), (4.0, 2.5, 0.00390625, '03', 0.51196172248803828), (4.0, 2.5, 0.00390625, '04', 0.52213114754098355), (4.0, 2.5, 32.0, '06', 0.51866883116883122), (4.0, 2.5, 0.0001220703125, '07', 0.5043478260869565), (4.0, 2.5, 0.0078125, '08', 0.51441441441441438), (4.0, 2.5, 0.125, '09', 0.52100161550888535), (4.0, 2.5, 0.03125, '10', 0.51552287581699341), (4.0, 2.5, 0.00390625, '11', 0.55231560891938247), (4.0, 2.5, 0.001953125, '12', 0.52003484320557491), (4.0, 2.5, 4096.0, '13', 0.50082644628099171), (4.0, 2.5, 0.015625, '14', 0.54676870748299322), (4.0, 2.5, 0.015625, '15', 0.52159468438538203), (4.0, 2.5, 0.001953125, '16', 0.53187919463087252), (4.0, 2.5, 0.0009765625, '17', 0.50334448160535117), (4.0, 2.5, 0.00390625, '18', 0.54063018242122718), (4.0, 2.5, 0.0078125, '19', 0.50661157024793391), (4.0, 2.5, 0.0078125, '20', 0.51782820097244731), (4.0, 3.0, 0.00048828125, '01', 0.50803858520900325), (4.0, 3.0, 32.0, '02', 0.5280898876404494), (4.0, 3.0, 0.0001220703125, '03', 0.50478468899521534), (4.0, 3.0, 8192.0, '04', 0.50983606557377048), (4.0, 3.0, 64.0, '06', 0.50974025974025972), (4.0, 3.0, 0.0001220703125, '07', 0.50173913043478258), (4.0, 3.0, 32.0, '08', 0.52522522522522519), (4.0, 3.0, 0.015625, '09', 0.51211631663974155), (4.0, 3.0, 0.015625, '10', 0.5106209150326797), (4.0, 3.0, 0.00390625, '11', 0.54202401372212694), (4.0, 3.0, 0.0009765625, '12', 0.51655052264808365), (4.0, 3.0, 64.0, '13', 0.5057851239669422), (4.0, 3.0, 0.0625, '14', 0.52976190476190477), (4.0, 3.0, 0.25, '15', 0.51079734219269102), (4.0, 3.0, 0.015625, '16', 0.52348993288590606), (4.0, 3.0, 0.0009765625, '17', 0.50250836120401343), (4.0, 3.0, 0.001953125, '18', 0.53150912106135983), (4.0, 3.0, 1024.0, '19', 0.50247933884297524), (4.0, 3.0, 128.0, '20', 0.51782820097244731), (4.0, 3.5, 0.0625, '01', 0.51527331189710612), (4.0, 3.5, 16.0, '02', 0.5321027287319422), (4.0, 3.5, 32.0, '03', 0.50877192982456143), (4.0, 3.5, 64.0, '04', 0.52049180327868849), (4.0, 3.5, 64.0, '06', 0.52029220779220775), (4.0, 3.5, 0.001953125, '07', 0.51217391304347826), (4.0, 3.5, 32.0, '08', 0.53963963963963968), (4.0, 3.5, 0.00390625, '09', 0.51857835218093695), (4.0, 3.5, 64.0, '10', 0.51388888888888884), (4.0, 3.5, 0.001953125, '11', 0.54631217838765012), (4.0, 3.5, 0.25, '12', 0.51829268292682928), (4.0, 3.5, 32.0, '13', 0.51239669421487599), (4.0, 3.5, 0.015625, '14', 0.54421768707482998), (4.0, 3.5, 0.125, '15', 0.51661129568106312), (4.0, 3.5, 16.0, '16', 0.53691275167785235), (4.0, 3.5, 0.0078125, '17', 0.51337792642140467), (4.0, 3.5, 0.0078125, '18', 0.53482587064676612), (4.0, 3.5, 4096.0, '19', 0.51570247933884295), (4.0, 3.5, 64.0, '20', 0.51863857374392219), (4.0, 4.0, 64.0, '01', 0.52572347266881025), (4.0, 4.0, 0.00390625, '02', 0.5337078651685393), (4.0, 4.0, 0.0625, '03', 0.51674641148325362), (4.0, 4.0, 64.0, '04', 0.51557377049180331), (4.0, 4.0, 32.0, '06', 0.51217532467532467), (4.0, 4.0, 0.001953125, '07', 0.52173913043478259), (4.0, 4.0, 32.0, '08', 0.53513513513513511), (4.0, 4.0, 4.0, '09', 0.52261712439418417), (4.0, 4.0, 64.0, '10', 0.51879084967320266), (4.0, 4.0, 0.00390625, '11', 0.55574614065180106), (4.0, 4.0, 4.0, '12', 0.53397212543554007), (4.0, 4.0, 64.0, '13', 0.5057851239669422), (4.0, 4.0, 16.0, '14', 0.54081632653061229), (4.0, 4.0, 0.015625, '15', 0.52076411960132896), (4.0, 4.0, 0.0078125, '16', 0.53020134228187921), (4.0, 4.0, 0.00390625, '17', 0.51337792642140467), (4.0, 4.0, 0.0078125, '18', 0.53316749585406298), (4.0, 4.0, 32.0, '19', 0.52314049586776856), (4.0, 4.0, 4.0, '20', 0.52836304700162073), (4.0, 4.5, 128.0, '01', 0.5313504823151125), (4.0, 4.5, 0.00390625, '02', 0.5337078651685393), (4.0, 4.5, 0.125, '03', 0.52073365231259972), (4.0, 4.5, 32.0, '04', 0.51721311475409837), (4.0, 4.5, 64.0, '06', 0.51623376623376627), (4.0, 4.5, 0.00390625, '07', 0.52782608695652178), (4.0, 4.5, 32.0, '08', 0.52792792792792798), (4.0, 4.5, 0.125, '09', 0.51938610662358642), (4.0, 4.5, 64.0, '10', 0.5326797385620915), (4.0, 4.5, 0.00390625, '11', 0.55917667238421953), (4.0, 4.5, 0.125, '12', 0.53135888501742157), (4.0, 4.5, 128.0, '13', 0.51322314049586781), (4.0, 4.5, 32.0, '14', 0.54506802721088432), (4.0, 4.5, 0.0078125, '15', 0.53405315614617943), (4.0, 4.5, 0.0078125, '16', 0.53187919463087252), (4.0, 4.5, 0.001953125, '17', 0.51421404682274252), (4.0, 4.5, 0.03125, '18', 0.53482587064676612), (4.0, 4.5, 32.0, '19', 0.52396694214876038), (4.0, 4.5, 32.0, '20', 0.53160453808752028)]
test_scores=[(1.0, 0.5, '01', 0.42307692307692307), (1.0, 0.5, '02', 0.54166666666666663), (1.0, 0.5, '03', 0.54545454545454541), (1.0, 0.5, '04', 0.66000000000000003), (1.0, 0.5, '06', 0.59375), (1.0, 0.5, '07', 0.55000000000000004), (1.0, 0.5, '08', 0.5625), (1.0, 0.5, '09', 0.51351351351351349), (1.0, 0.5, '10', 0.45588235294117646), (1.0, 0.5, '11', 0.39423076923076922), (1.0, 0.5, '12', 0.55737704918032782), (1.0, 0.5, '13', 0.41860465116279072), (1.0, 0.5, '14', 0.54255319148936165), (1.0, 0.5, '15', 0.46969696969696972), (1.0, 0.5, '16', 0.5), (1.0, 0.5, '17', 0.56756756756756754), (1.0, 0.5, '18', 0.59375), (1.0, 0.5, '19', 0.53333333333333333), (1.0, 0.5, '20', 0.47674418604651164), (1.0, 1.0, '01', 0.61538461538461542), (1.0, 1.0, '02', 0.45833333333333331), (1.0, 1.0, '03', 0.59090909090909094), (1.0, 1.0, '04', 0.78000000000000003), (1.0, 1.0, '06', 0.515625), (1.0, 1.0, '07', 0.54166666666666663), (1.0, 1.0, '08', 0.51249999999999996), (1.0, 1.0, '09', 0.47297297297297297), (1.0, 1.0, '10', 0.45588235294117646), (1.0, 1.0, '11', 0.47115384615384615), (1.0, 1.0, '12', 0.49180327868852458), (1.0, 1.0, '13', 0.48837209302325579), (1.0, 1.0, '14', 0.46808510638297873), (1.0, 1.0, '15', 0.5), (1.0, 1.0, '16', 0.5641025641025641), (1.0, 1.0, '17', 0.60810810810810811), (1.0, 1.0, '18', 0.390625), (1.0, 1.0, '19', 0.53333333333333333), (1.0, 1.0, '20', 0.48837209302325579), (1.0, 1.5, '01', 0.69230769230769229), (1.0, 1.5, '02', 0.45833333333333331), (1.0, 1.5, '03', 0.45454545454545453), (1.0, 1.5, '04', 0.46000000000000002), (1.0, 1.5, '06', 0.5625), (1.0, 1.5, '07', 0.53333333333333333), (1.0, 1.5, '08', 0.46250000000000002), (1.0, 1.5, '09', 0.5), (1.0, 1.5, '10', 0.44117647058823528), (1.0, 1.5, '11', 0.5), (1.0, 1.5, '12', 0.58196721311475408), (1.0, 1.5, '13', 0.5), (1.0, 1.5, '14', 0.38297872340425532), (1.0, 1.5, '15', 0.45454545454545453), (1.0, 1.5, '16', 0.53846153846153844), (1.0, 1.5, '17', 0.48648648648648651), (1.0, 1.5, '18', 0.46875), (1.0, 1.5, '19', 0.48333333333333334), (1.0, 1.5, '20', 0.41860465116279072), (1.0, 2.0, '01', 0.65384615384615385), (1.0, 2.0, '02', 0.45833333333333331), (1.0, 2.0, '03', 0.5), (1.0, 2.0, '04', 0.52000000000000002), (1.0, 2.0, '06', 0.515625), (1.0, 2.0, '07', 0.51666666666666672), (1.0, 2.0, '08', 0.4375), (1.0, 2.0, '09', 0.48648648648648651), (1.0, 2.0, '10', 0.4264705882352941), (1.0, 2.0, '11', 0.49038461538461536), (1.0, 2.0, '12', 0.54918032786885251), (1.0, 2.0, '13', 0.56976744186046513), (1.0, 2.0, '14', 0.34042553191489361), (1.0, 2.0, '15', 0.45454545454545453), (1.0, 2.0, '16', 0.5641025641025641), (1.0, 2.0, '17', 0.58108108108108103), (1.0, 2.0, '18', 0.5), (1.0, 2.0, '19', 0.51666666666666672), (1.0, 2.0, '20', 0.53488372093023251), (1.0, 2.5, '01', 0.57692307692307687), (1.0, 2.5, '02', 0.54166666666666663), (1.0, 2.5, '03', 0.63636363636363635), (1.0, 2.5, '04', 0.5), (1.0, 2.5, '06', 0.453125), (1.0, 2.5, '07', 0.47499999999999998), (1.0, 2.5, '08', 0.48749999999999999), (1.0, 2.5, '09', 0.51351351351351349), (1.0, 2.5, '10', 0.44117647058823528), (1.0, 2.5, '11', 0.51923076923076927), (1.0, 2.5, '12', 0.52459016393442626), (1.0, 2.5, '13', 0.53488372093023251), (1.0, 2.5, '14', 0.38297872340425532), (1.0, 2.5, '15', 0.45454545454545453), (1.0, 2.5, '16', 0.62820512820512819), (1.0, 2.5, '17', 0.48648648648648651), (1.0, 2.5, '18', 0.484375), (1.0, 2.5, '19', 0.5), (1.0, 2.5, '20', 0.54651162790697672), (1.0, 3.0, '01', 0.53846153846153844), (1.0, 3.0, '02', 0.5), (1.0, 3.0, '03', 0.59090909090909094), (1.0, 3.0, '04', 0.54000000000000004), (1.0, 3.0, '06', 0.421875), (1.0, 3.0, '07', 0.5), (1.0, 3.0, '08', 0.45624999999999999), (1.0, 3.0, '09', 0.56756756756756754), (1.0, 3.0, '10', 0.41176470588235292), (1.0, 3.0, '11', 0.51923076923076927), (1.0, 3.0, '12', 0.54098360655737709), (1.0, 3.0, '13', 0.52325581395348841), (1.0, 3.0, '14', 0.35106382978723405), (1.0, 3.0, '15', 0.51515151515151514), (1.0, 3.0, '16', 0.53846153846153844), (1.0, 3.0, '17', 0.47297297297297297), (1.0, 3.0, '18', 0.5), (1.0, 3.0, '19', 0.53333333333333333), (1.0, 3.0, '20', 0.61627906976744184), (1.0, 3.5, '01', 0.57692307692307687), (1.0, 3.5, '02', 0.5), (1.0, 3.5, '03', 0.63636363636363635), (1.0, 3.5, '04', 0.44), (1.0, 3.5, '06', 0.515625), (1.0, 3.5, '07', 0.5), (1.0, 3.5, '08', 0.45000000000000001), (1.0, 3.5, '09', 0.45945945945945948), (1.0, 3.5, '10', 0.3235294117647059), (1.0, 3.5, '11', 0.48076923076923078), (1.0, 3.5, '12', 0.52459016393442626), (1.0, 3.5, '13', 0.46511627906976744), (1.0, 3.5, '14', 0.32978723404255317), (1.0, 3.5, '15', 0.48484848484848486), (1.0, 3.5, '16', 0.5641025641025641), (1.0, 3.5, '17', 0.40540540540540543), (1.0, 3.5, '18', 0.515625), (1.0, 3.5, '19', 0.38333333333333336), (1.0, 3.5, '20', 0.60465116279069764), (1.0, 4.0, '01', 0.5), (1.0, 4.0, '02', 0.54166666666666663), (1.0, 4.0, '03', 0.77272727272727271), (1.0, 4.0, '04', 0.62), (1.0, 4.0, '06', 0.484375), (1.0, 4.0, '07', 0.5), (1.0, 4.0, '08', 0.44374999999999998), (1.0, 4.0, '09', 0.44594594594594594), (1.0, 4.0, '10', 0.47058823529411764), (1.0, 4.0, '11', 0.47115384615384615), (1.0, 4.0, '12', 0.51639344262295084), (1.0, 4.0, '13', 0.44186046511627908), (1.0, 4.0, '14', 0.37234042553191488), (1.0, 4.0, '15', 0.51515151515151514), (1.0, 4.0, '16', 0.55128205128205132), (1.0, 4.0, '17', 0.47297297297297297), (1.0, 4.0, '18', 0.515625), (1.0, 4.0, '19', 0.53333333333333333), (1.0, 4.0, '20', 0.63953488372093026), (1.0, 4.5, '01', 0.38461538461538464), (1.0, 4.5, '02', 0.54166666666666663), (1.0, 4.5, '03', 0.72727272727272729), (1.0, 4.5, '04', 0.46000000000000002), (1.0, 4.5, '06', 0.5), (1.0, 4.5, '07', 0.54166666666666663), (1.0, 4.5, '08', 0.36249999999999999), (1.0, 4.5, '09', 0.44594594594594594), (1.0, 4.5, '10', 0.47058823529411764), (1.0, 4.5, '11', 0.44230769230769229), (1.0, 4.5, '12', 0.51639344262295084), (1.0, 4.5, '13', 0.45348837209302323), (1.0, 4.5, '14', 0.36170212765957449), (1.0, 4.5, '15', 0.54545454545454541), (1.0, 4.5, '16', 0.66666666666666663), (1.0, 4.5, '17', 0.41891891891891891), (1.0, 4.5, '18', 0.515625), (1.0, 4.5, '19', 0.45000000000000001), (1.0, 4.5, '20', 0.56976744186046513), (1.5, 0.5, '01', 0.53846153846153844), (1.5, 0.5, '02', 0.54166666666666663), (1.5, 0.5, '03', 0.31818181818181818), (1.5, 0.5, '04', 0.54000000000000004), (1.5, 0.5, '06', 0.578125), (1.5, 0.5, '07', 0.52500000000000002), (1.5, 0.5, '08', 0.51875000000000004), (1.5, 0.5, '09', 0.40540540540540543), (1.5, 0.5, '10', 0.54411764705882348), (1.5, 0.5, '11', 0.46153846153846156), (1.5, 0.5, '12', 0.55737704918032782), (1.5, 0.5, '13', 0.55813953488372092), (1.5, 0.5, '14', 0.46808510638297873), (1.5, 0.5, '15', 0.46969696969696972), (1.5, 0.5, '16', 0.48717948717948717), (1.5, 0.5, '17', 0.47297297297297297), (1.5, 0.5, '18', 0.484375), (1.5, 0.5, '19', 0.51666666666666672), (1.5, 0.5, '20', 0.53488372093023251), (1.5, 1.0, '01', 0.46153846153846156), (1.5, 1.0, '02', 0.45833333333333331), (1.5, 1.0, '03', 0.40909090909090912), (1.5, 1.0, '04', 0.54000000000000004), (1.5, 1.0, '06', 0.671875), (1.5, 1.0, '07', 0.59999999999999998), (1.5, 1.0, '08', 0.625), (1.5, 1.0, '09', 0.40540540540540543), (1.5, 1.0, '10', 0.51470588235294112), (1.5, 1.0, '11', 0.50961538461538458), (1.5, 1.0, '12', 0.45901639344262296), (1.5, 1.0, '13', 0.51162790697674421), (1.5, 1.0, '14', 0.5), (1.5, 1.0, '15', 0.53030303030303028), (1.5, 1.0, '16', 0.57692307692307687), (1.5, 1.0, '17', 0.54054054054054057), (1.5, 1.0, '18', 0.421875), (1.5, 1.0, '19', 0.45000000000000001), (1.5, 1.0, '20', 0.45348837209302323), (1.5, 1.5, '01', 0.57692307692307687), (1.5, 1.5, '02', 0.375), (1.5, 1.5, '03', 0.22727272727272727), (1.5, 1.5, '04', 0.44), (1.5, 1.5, '06', 0.546875), (1.5, 1.5, '07', 0.54166666666666663), (1.5, 1.5, '08', 0.51875000000000004), (1.5, 1.5, '09', 0.44594594594594594), (1.5, 1.5, '10', 0.54411764705882348), (1.5, 1.5, '11', 0.53846153846153844), (1.5, 1.5, '12', 0.46721311475409838), (1.5, 1.5, '13', 0.51162790697674421), (1.5, 1.5, '14', 0.53191489361702127), (1.5, 1.5, '15', 0.43939393939393939), (1.5, 1.5, '16', 0.47435897435897434), (1.5, 1.5, '17', 0.55405405405405406), (1.5, 1.5, '18', 0.46875), (1.5, 1.5, '19', 0.5), (1.5, 1.5, '20', 0.61627906976744184), (1.5, 2.0, '01', 0.61538461538461542), (1.5, 2.0, '02', 0.375), (1.5, 2.0, '03', 0.59090909090909094), (1.5, 2.0, '04', 0.56000000000000005), (1.5, 2.0, '06', 0.5625), (1.5, 2.0, '07', 0.59166666666666667), (1.5, 2.0, '08', 0.51875000000000004), (1.5, 2.0, '09', 0.58108108108108103), (1.5, 2.0, '10', 0.5), (1.5, 2.0, '11', 0.50961538461538458), (1.5, 2.0, '12', 0.48360655737704916), (1.5, 2.0, '13', 0.55813953488372092), (1.5, 2.0, '14', 0.40425531914893614), (1.5, 2.0, '15', 0.42424242424242425), (1.5, 2.0, '16', 0.48717948717948717), (1.5, 2.0, '17', 0.36486486486486486), (1.5, 2.0, '18', 0.515625), (1.5, 2.0, '19', 0.45000000000000001), (1.5, 2.0, '20', 0.52325581395348841), (1.5, 2.5, '01', 0.53846153846153844), (1.5, 2.5, '02', 0.375), (1.5, 2.5, '03', 0.63636363636363635), (1.5, 2.5, '04', 0.52000000000000002), (1.5, 2.5, '06', 0.421875), (1.5, 2.5, '07', 0.5083333333333333), (1.5, 2.5, '08', 0.55625000000000002), (1.5, 2.5, '09', 0.51351351351351349), (1.5, 2.5, '10', 0.4264705882352941), (1.5, 2.5, '11', 0.48076923076923078), (1.5, 2.5, '12', 0.46721311475409838), (1.5, 2.5, '13', 0.61627906976744184), (1.5, 2.5, '14', 0.40425531914893614), (1.5, 2.5, '15', 0.42424242424242425), (1.5, 2.5, '16', 0.47435897435897434), (1.5, 2.5, '17', 0.39189189189189189), (1.5, 2.5, '18', 0.5), (1.5, 2.5, '19', 0.46666666666666667), (1.5, 2.5, '20', 0.52325581395348841), (1.5, 3.0, '01', 0.57692307692307687), (1.5, 3.0, '02', 0.54166666666666663), (1.5, 3.0, '03', 0.72727272727272729), (1.5, 3.0, '04', 0.56000000000000005), (1.5, 3.0, '06', 0.5625), (1.5, 3.0, '07', 0.53333333333333333), (1.5, 3.0, '08', 0.44374999999999998), (1.5, 3.0, '09', 0.51351351351351349), (1.5, 3.0, '10', 0.5), (1.5, 3.0, '11', 0.47115384615384615), (1.5, 3.0, '12', 0.5), (1.5, 3.0, '13', 0.58139534883720934), (1.5, 3.0, '14', 0.47872340425531917), (1.5, 3.0, '15', 0.34848484848484851), (1.5, 3.0, '16', 0.44871794871794873), (1.5, 3.0, '17', 0.39189189189189189), (1.5, 3.0, '18', 0.484375), (1.5, 3.0, '19', 0.41666666666666669), (1.5, 3.0, '20', 0.39534883720930231), (1.5, 3.5, '01', 0.53846153846153844), (1.5, 3.5, '02', 0.5), (1.5, 3.5, '03', 0.63636363636363635), (1.5, 3.5, '04', 0.54000000000000004), (1.5, 3.5, '06', 0.5), (1.5, 3.5, '07', 0.53333333333333333), (1.5, 3.5, '08', 0.45000000000000001), (1.5, 3.5, '09', 0.51351351351351349), (1.5, 3.5, '10', 0.48529411764705882), (1.5, 3.5, '11', 0.51923076923076927), (1.5, 3.5, '12', 0.50819672131147542), (1.5, 3.5, '13', 0.54651162790697672), (1.5, 3.5, '14', 0.38297872340425532), (1.5, 3.5, '15', 0.36363636363636365), (1.5, 3.5, '16', 0.48717948717948717), (1.5, 3.5, '17', 0.36486486486486486), (1.5, 3.5, '18', 0.46875), (1.5, 3.5, '19', 0.38333333333333336), (1.5, 3.5, '20', 0.44186046511627908), (1.5, 4.0, '01', 0.57692307692307687), (1.5, 4.0, '02', 0.54166666666666663), (1.5, 4.0, '03', 0.68181818181818177), (1.5, 4.0, '04', 0.57999999999999996), (1.5, 4.0, '06', 0.5625), (1.5, 4.0, '07', 0.48333333333333334), (1.5, 4.0, '08', 0.42499999999999999), (1.5, 4.0, '09', 0.51351351351351349), (1.5, 4.0, '10', 0.57352941176470584), (1.5, 4.0, '11', 0.52884615384615385), (1.5, 4.0, '12', 0.54098360655737709), (1.5, 4.0, '13', 0.54651162790697672), (1.5, 4.0, '14', 0.35106382978723405), (1.5, 4.0, '15', 0.34848484848484851), (1.5, 4.0, '16', 0.42307692307692307), (1.5, 4.0, '17', 0.45945945945945948), (1.5, 4.0, '18', 0.5), (1.5, 4.0, '19', 0.46666666666666667), (1.5, 4.0, '20', 0.47674418604651164), (1.5, 4.5, '01', 0.57692307692307687), (1.5, 4.5, '02', 0.54166666666666663), (1.5, 4.5, '03', 0.77272727272727271), (1.5, 4.5, '04', 0.5), (1.5, 4.5, '06', 0.5625), (1.5, 4.5, '07', 0.55000000000000004), (1.5, 4.5, '08', 0.43125000000000002), (1.5, 4.5, '09', 0.52702702702702697), (1.5, 4.5, '10', 0.52941176470588236), (1.5, 4.5, '11', 0.52884615384615385), (1.5, 4.5, '12', 0.5), (1.5, 4.5, '13', 0.63953488372093026), (1.5, 4.5, '14', 0.39361702127659576), (1.5, 4.5, '15', 0.34848484848484851), (1.5, 4.5, '16', 0.46153846153846156), (1.5, 4.5, '17', 0.3783783783783784), (1.5, 4.5, '18', 0.515625), (1.5, 4.5, '19', 0.34999999999999998), (1.5, 4.5, '20', 0.48837209302325579), (2.0, 0.5, '01', 0.5), (2.0, 0.5, '02', 0.5), (2.0, 0.5, '03', 0.5), (2.0, 0.5, '04', 0.5), (2.0, 0.5, '06', 0.5), (2.0, 0.5, '07', 0.5), (2.0, 0.5, '08', 0.5), (2.0, 0.5, '09', 0.43243243243243246), (2.0, 0.5, '10', 0.41176470588235292), (2.0, 0.5, '11', 0.47115384615384615), (2.0, 0.5, '12', 0.5), (2.0, 0.5, '13', 0.45348837209302323), (2.0, 0.5, '14', 0.5), (2.0, 0.5, '15', 0.5), (2.0, 0.5, '16', 0.5), (2.0, 0.5, '17', 0.35135135135135137), (2.0, 0.5, '18', 0.5), (2.0, 0.5, '19', 0.5), (2.0, 0.5, '20', 0.44186046511627908), (2.0, 1.0, '01', 0.5), (2.0, 1.0, '02', 0.54166666666666663), (2.0, 1.0, '03', 0.5), (2.0, 1.0, '04', 0.57999999999999996), (2.0, 1.0, '06', 0.5), (2.0, 1.0, '07', 0.5), (2.0, 1.0, '08', 0.5), (2.0, 1.0, '09', 0.47297297297297297), (2.0, 1.0, '10', 0.52941176470588236), (2.0, 1.0, '11', 0.49038461538461536), (2.0, 1.0, '12', 0.5), (2.0, 1.0, '13', 0.45348837209302323), (2.0, 1.0, '14', 0.38297872340425532), (2.0, 1.0, '15', 0.5757575757575758), (2.0, 1.0, '16', 0.5), (2.0, 1.0, '17', 0.60810810810810811), (2.0, 1.0, '18', 0.53125), (2.0, 1.0, '19', 0.51666666666666672), (2.0, 1.0, '20', 0.52325581395348841), (2.0, 1.5, '01', 0.5), (2.0, 1.5, '02', 0.45833333333333331), (2.0, 1.5, '03', 0.5), (2.0, 1.5, '04', 0.54000000000000004), (2.0, 1.5, '06', 0.59375), (2.0, 1.5, '07', 0.55833333333333335), (2.0, 1.5, '08', 0.45624999999999999), (2.0, 1.5, '09', 0.28378378378378377), (2.0, 1.5, '10', 0.5), (2.0, 1.5, '11', 0.50961538461538458), (2.0, 1.5, '12', 0.49180327868852458), (2.0, 1.5, '13', 0.5), (2.0, 1.5, '14', 0.44680851063829785), (2.0, 1.5, '15', 0.5), (2.0, 1.5, '16', 0.58974358974358976), (2.0, 1.5, '17', 0.47297297297297297), (2.0, 1.5, '18', 0.453125), (2.0, 1.5, '19', 0.53333333333333333), (2.0, 1.5, '20', 0.58139534883720934), (2.0, 2.0, '01', 0.57692307692307687), (2.0, 2.0, '02', 0.45833333333333331), (2.0, 2.0, '03', 0.45454545454545453), (2.0, 2.0, '04', 0.47999999999999998), (2.0, 2.0, '06', 0.609375), (2.0, 2.0, '07', 0.55833333333333335), (2.0, 2.0, '08', 0.41875000000000001), (2.0, 2.0, '09', 0.43243243243243246), (2.0, 2.0, '10', 0.5), (2.0, 2.0, '11', 0.57692307692307687), (2.0, 2.0, '12', 0.49180327868852458), (2.0, 2.0, '13', 0.41860465116279072), (2.0, 2.0, '14', 0.39361702127659576), (2.0, 2.0, '15', 0.5), (2.0, 2.0, '16', 0.51282051282051277), (2.0, 2.0, '17', 0.45945945945945948), (2.0, 2.0, '18', 0.4375), (2.0, 2.0, '19', 0.55000000000000004), (2.0, 2.0, '20', 0.55813953488372092), (2.0, 2.5, '01', 0.53846153846153844), (2.0, 2.5, '02', 0.5), (2.0, 2.5, '03', 0.5), (2.0, 2.5, '04', 0.41999999999999998), (2.0, 2.5, '06', 0.53125), (2.0, 2.5, '07', 0.54166666666666663), (2.0, 2.5, '08', 0.5), (2.0, 2.5, '09', 0.52702702702702697), (2.0, 2.5, '10', 0.5), (2.0, 2.5, '11', 0.50961538461538458), (2.0, 2.5, '12', 0.53278688524590168), (2.0, 2.5, '13', 0.38372093023255816), (2.0, 2.5, '14', 0.36170212765957449), (2.0, 2.5, '15', 0.40909090909090912), (2.0, 2.5, '16', 0.57692307692307687), (2.0, 2.5, '17', 0.39189189189189189), (2.0, 2.5, '18', 0.484375), (2.0, 2.5, '19', 0.5), (2.0, 2.5, '20', 0.52325581395348841), (2.0, 3.0, '01', 0.53846153846153844), (2.0, 3.0, '02', 0.375), (2.0, 3.0, '03', 0.68181818181818177), (2.0, 3.0, '04', 0.41999999999999998), (2.0, 3.0, '06', 0.53125), (2.0, 3.0, '07', 0.55833333333333335), (2.0, 3.0, '08', 0.42499999999999999), (2.0, 3.0, '09', 0.33783783783783783), (2.0, 3.0, '10', 0.55882352941176472), (2.0, 3.0, '11', 0.56730769230769229), (2.0, 3.0, '12', 0.52459016393442626), (2.0, 3.0, '13', 0.44186046511627908), (2.0, 3.0, '14', 0.37234042553191488), (2.0, 3.0, '15', 0.43939393939393939), (2.0, 3.0, '16', 0.60256410256410253), (2.0, 3.0, '17', 0.40540540540540543), (2.0, 3.0, '18', 0.515625), (2.0, 3.0, '19', 0.56666666666666665), (2.0, 3.0, '20', 0.55813953488372092), (2.0, 3.5, '01', 0.53846153846153844), (2.0, 3.5, '02', 0.375), (2.0, 3.5, '03', 0.54545454545454541), (2.0, 3.5, '04', 0.62), (2.0, 3.5, '06', 0.609375), (2.0, 3.5, '07', 0.57499999999999996), (2.0, 3.5, '08', 0.41249999999999998), (2.0, 3.5, '09', 0.5), (2.0, 3.5, '10', 0.48529411764705882), (2.0, 3.5, '11', 0.56730769230769229), (2.0, 3.5, '12', 0.52459016393442626), (2.0, 3.5, '13', 0.45348837209302323), (2.0, 3.5, '14', 0.42553191489361702), (2.0, 3.5, '15', 0.48484848484848486), (2.0, 3.5, '16', 0.48717948717948717), (2.0, 3.5, '17', 0.3783783783783784), (2.0, 3.5, '18', 0.5), (2.0, 3.5, '19', 0.55000000000000004), (2.0, 3.5, '20', 0.58139534883720934), (2.0, 4.0, '01', 0.5), (2.0, 4.0, '02', 0.375), (2.0, 4.0, '03', 0.68181818181818177), (2.0, 4.0, '04', 0.54000000000000004), (2.0, 4.0, '06', 0.515625), (2.0, 4.0, '07', 0.65000000000000002), (2.0, 4.0, '08', 0.4375), (2.0, 4.0, '09', 0.43243243243243246), (2.0, 4.0, '10', 0.48529411764705882), (2.0, 4.0, '11', 0.55769230769230771), (2.0, 4.0, '12', 0.50819672131147542), (2.0, 4.0, '13', 0.53488372093023251), (2.0, 4.0, '14', 0.37234042553191488), (2.0, 4.0, '15', 0.45454545454545453), (2.0, 4.0, '16', 0.5641025641025641), (2.0, 4.0, '17', 0.43243243243243246), (2.0, 4.0, '18', 0.5), (2.0, 4.0, '19', 0.56666666666666665), (2.0, 4.0, '20', 0.48837209302325579), (2.0, 4.5, '01', 0.5), (2.0, 4.5, '02', 0.41666666666666669), (2.0, 4.5, '03', 0.77272727272727271), (2.0, 4.5, '04', 0.71999999999999997), (2.0, 4.5, '06', 0.546875), (2.0, 4.5, '07', 0.59999999999999998), (2.0, 4.5, '08', 0.45624999999999999), (2.0, 4.5, '09', 0.43243243243243246), (2.0, 4.5, '10', 0.48529411764705882), (2.0, 4.5, '11', 0.50961538461538458), (2.0, 4.5, '12', 0.48360655737704916), (2.0, 4.5, '13', 0.44186046511627908), (2.0, 4.5, '14', 0.36170212765957449), (2.0, 4.5, '15', 0.43939393939393939), (2.0, 4.5, '16', 0.51282051282051277), (2.0, 4.5, '17', 0.48648648648648651), (2.0, 4.5, '18', 0.5), (2.0, 4.5, '19', 0.46666666666666667), (2.0, 4.5, '20', 0.43023255813953487), (2.5, 0.5, '01', 0.57692307692307687), (2.5, 0.5, '02', 0.66666666666666663), (2.5, 0.5, '03', 0.63636363636363635), (2.5, 0.5, '04', 0.41999999999999998), (2.5, 0.5, '06', 0.546875), (2.5, 0.5, '07', 0.52500000000000002), (2.5, 0.5, '08', 0.53125), (2.5, 0.5, '09', 0.35135135135135137), (2.5, 0.5, '10', 0.38235294117647056), (2.5, 0.5, '11', 0.57692307692307687), (2.5, 0.5, '12', 0.49180327868852458), (2.5, 0.5, '13', 0.60465116279069764), (2.5, 0.5, '14', 0.55319148936170215), (2.5, 0.5, '15', 0.53030303030303028), (2.5, 0.5, '16', 0.47435897435897434), (2.5, 0.5, '17', 0.3783783783783784), (2.5, 0.5, '18', 0.46875), (2.5, 0.5, '19', 0.45000000000000001), (2.5, 0.5, '20', 0.62790697674418605), (2.5, 1.0, '01', 0.61538461538461542), (2.5, 1.0, '02', 0.45833333333333331), (2.5, 1.0, '03', 0.5), (2.5, 1.0, '04', 0.52000000000000002), (2.5, 1.0, '06', 0.40625), (2.5, 1.0, '07', 0.55833333333333335), (2.5, 1.0, '08', 0.48125000000000001), (2.5, 1.0, '09', 0.40540540540540543), (2.5, 1.0, '10', 0.57352941176470584), (2.5, 1.0, '11', 0.56730769230769229), (2.5, 1.0, '12', 0.5), (2.5, 1.0, '13', 0.46511627906976744), (2.5, 1.0, '14', 0.41489361702127658), (2.5, 1.0, '15', 0.53030303030303028), (2.5, 1.0, '16', 0.5641025641025641), (2.5, 1.0, '17', 0.51351351351351349), (2.5, 1.0, '18', 0.453125), (2.5, 1.0, '19', 0.48333333333333334), (2.5, 1.0, '20', 0.5), (2.5, 1.5, '01', 0.69230769230769229), (2.5, 1.5, '02', 0.33333333333333331), (2.5, 1.5, '03', 0.31818181818181818), (2.5, 1.5, '04', 0.59999999999999998), (2.5, 1.5, '06', 0.5625), (2.5, 1.5, '07', 0.56666666666666665), (2.5, 1.5, '08', 0.53125), (2.5, 1.5, '09', 0.43243243243243246), (2.5, 1.5, '10', 0.6029411764705882), (2.5, 1.5, '11', 0.53846153846153844), (2.5, 1.5, '12', 0.49180327868852458), (2.5, 1.5, '13', 0.43023255813953487), (2.5, 1.5, '14', 0.41489361702127658), (2.5, 1.5, '15', 0.56060606060606055), (2.5, 1.5, '16', 0.60256410256410253), (2.5, 1.5, '17', 0.52702702702702697), (2.5, 1.5, '18', 0.359375), (2.5, 1.5, '19', 0.46666666666666667), (2.5, 1.5, '20', 0.55813953488372092), (2.5, 2.0, '01', 0.61538461538461542), (2.5, 2.0, '02', 0.41666666666666669), (2.5, 2.0, '03', 0.45454545454545453), (2.5, 2.0, '04', 0.57999999999999996), (2.5, 2.0, '06', 0.453125), (2.5, 2.0, '07', 0.58333333333333337), (2.5, 2.0, '08', 0.54374999999999996), (2.5, 2.0, '09', 0.39189189189189189), (2.5, 2.0, '10', 0.61764705882352944), (2.5, 2.0, '11', 0.55769230769230771), (2.5, 2.0, '12', 0.53278688524590168), (2.5, 2.0, '13', 0.52325581395348841), (2.5, 2.0, '14', 0.41489361702127658), (2.5, 2.0, '15', 0.60606060606060608), (2.5, 2.0, '16', 0.61538461538461542), (2.5, 2.0, '17', 0.47297297297297297), (2.5, 2.0, '18', 0.375), (2.5, 2.0, '19', 0.51666666666666672), (2.5, 2.0, '20', 0.55813953488372092), (2.5, 2.5, '01', 0.65384615384615385), (2.5, 2.5, '02', 0.5), (2.5, 2.5, '03', 0.5), (2.5, 2.5, '04', 0.40000000000000002), (2.5, 2.5, '06', 0.578125), (2.5, 2.5, '07', 0.52500000000000002), (2.5, 2.5, '08', 0.48749999999999999), (2.5, 2.5, '09', 0.35135135135135137), (2.5, 2.5, '10', 0.52941176470588236), (2.5, 2.5, '11', 0.57692307692307687), (2.5, 2.5, '12', 0.42622950819672129), (2.5, 2.5, '13', 0.48837209302325579), (2.5, 2.5, '14', 0.43617021276595747), (2.5, 2.5, '15', 0.53030303030303028), (2.5, 2.5, '16', 0.60256410256410253), (2.5, 2.5, '17', 0.54054054054054057), (2.5, 2.5, '18', 0.4375), (2.5, 2.5, '19', 0.48333333333333334), (2.5, 2.5, '20', 0.54651162790697672), (2.5, 3.0, '01', 0.53846153846153844), (2.5, 3.0, '02', 0.5), (2.5, 3.0, '03', 0.5), (2.5, 3.0, '04', 0.29999999999999999), (2.5, 3.0, '06', 0.578125), (2.5, 3.0, '07', 0.54166666666666663), (2.5, 3.0, '08', 0.4375), (2.5, 3.0, '09', 0.36486486486486486), (2.5, 3.0, '10', 0.5), (2.5, 3.0, '11', 0.54807692307692313), (2.5, 3.0, '12', 0.53278688524590168), (2.5, 3.0, '13', 0.43023255813953487), (2.5, 3.0, '14', 0.41489361702127658), (2.5, 3.0, '15', 0.48484848484848486), (2.5, 3.0, '16', 0.58974358974358976), (2.5, 3.0, '17', 0.48648648648648651), (2.5, 3.0, '18', 0.5), (2.5, 3.0, '19', 0.45000000000000001), (2.5, 3.0, '20', 0.48837209302325579), (2.5, 3.5, '01', 0.61538461538461542), (2.5, 3.5, '02', 0.41666666666666669), (2.5, 3.5, '03', 0.54545454545454541), (2.5, 3.5, '04', 0.26000000000000001), (2.5, 3.5, '06', 0.46875), (2.5, 3.5, '07', 0.55000000000000004), (2.5, 3.5, '08', 0.41249999999999998), (2.5, 3.5, '09', 0.43243243243243246), (2.5, 3.5, '10', 0.47058823529411764), (2.5, 3.5, '11', 0.51923076923076927), (2.5, 3.5, '12', 0.54098360655737709), (2.5, 3.5, '13', 0.45348837209302323), (2.5, 3.5, '14', 0.40425531914893614), (2.5, 3.5, '15', 0.5), (2.5, 3.5, '16', 0.64102564102564108), (2.5, 3.5, '17', 0.54054054054054057), (2.5, 3.5, '18', 0.46875), (2.5, 3.5, '19', 0.53333333333333333), (2.5, 3.5, '20', 0.53488372093023251), (2.5, 4.0, '01', 0.46153846153846156), (2.5, 4.0, '02', 0.45833333333333331), (2.5, 4.0, '03', 0.59090909090909094), (2.5, 4.0, '04', 0.35999999999999999), (2.5, 4.0, '06', 0.46875), (2.5, 4.0, '07', 0.5083333333333333), (2.5, 4.0, '08', 0.36875000000000002), (2.5, 4.0, '09', 0.43243243243243246), (2.5, 4.0, '10', 0.55882352941176472), (2.5, 4.0, '11', 0.5), (2.5, 4.0, '12', 0.52459016393442626), (2.5, 4.0, '13', 0.47674418604651164), (2.5, 4.0, '14', 0.38297872340425532), (2.5, 4.0, '15', 0.5), (2.5, 4.0, '16', 0.64102564102564108), (2.5, 4.0, '17', 0.58108108108108103), (2.5, 4.0, '18', 0.5), (2.5, 4.0, '19', 0.45000000000000001), (2.5, 4.0, '20', 0.5), (2.5, 4.5, '01', 0.53846153846153844), (2.5, 4.5, '02', 0.58333333333333337), (2.5, 4.5, '03', 0.54545454545454541), (2.5, 4.5, '04', 0.35999999999999999), (2.5, 4.5, '06', 0.5625), (2.5, 4.5, '07', 0.52500000000000002), (2.5, 4.5, '08', 0.38750000000000001), (2.5, 4.5, '09', 0.33783783783783783), (2.5, 4.5, '10', 0.54411764705882348), (2.5, 4.5, '11', 0.47115384615384615), (2.5, 4.5, '12', 0.47540983606557374), (2.5, 4.5, '13', 0.51162790697674421), (2.5, 4.5, '14', 0.40425531914893614), (2.5, 4.5, '15', 0.60606060606060608), (2.5, 4.5, '16', 0.66666666666666663), (2.5, 4.5, '17', 0.54054054054054057), (2.5, 4.5, '18', 0.5), (2.5, 4.5, '19', 0.43333333333333335), (2.5, 4.5, '20', 0.47674418604651164), (3.0, 0.5, '01', 0.34615384615384615), (3.0, 0.5, '02', 0.58333333333333337), (3.0, 0.5, '03', 0.45454545454545453), (3.0, 0.5, '04', 0.54000000000000004), (3.0, 0.5, '06', 0.59375), (3.0, 0.5, '07', 0.45000000000000001), (3.0, 0.5, '08', 0.5), (3.0, 0.5, '09', 0.58108108108108103), (3.0, 0.5, '10', 0.57352941176470584), (3.0, 0.5, '11', 0.60576923076923073), (3.0, 0.5, '12', 0.54918032786885251), (3.0, 0.5, '13', 0.48837209302325579), (3.0, 0.5, '14', 0.51063829787234039), (3.0, 0.5, '15', 0.56060606060606055), (3.0, 0.5, '16', 0.35897435897435898), (3.0, 0.5, '17', 0.43243243243243246), (3.0, 0.5, '18', 0.453125), (3.0, 0.5, '19', 0.48333333333333334), (3.0, 0.5, '20', 0.39534883720930231), (3.0, 1.0, '01', 0.53846153846153844), (3.0, 1.0, '02', 0.66666666666666663), (3.0, 1.0, '03', 0.59090909090909094), (3.0, 1.0, '04', 0.47999999999999998), (3.0, 1.0, '06', 0.640625), (3.0, 1.0, '07', 0.51666666666666672), (3.0, 1.0, '08', 0.51875000000000004), (3.0, 1.0, '09', 0.45945945945945948), (3.0, 1.0, '10', 0.54411764705882348), (3.0, 1.0, '11', 0.57692307692307687), (3.0, 1.0, '12', 0.54918032786885251), (3.0, 1.0, '13', 0.56976744186046513), (3.0, 1.0, '14', 0.46808510638297873), (3.0, 1.0, '15', 0.48484848484848486), (3.0, 1.0, '16', 0.33333333333333331), (3.0, 1.0, '17', 0.55405405405405406), (3.0, 1.0, '18', 0.53125), (3.0, 1.0, '19', 0.55000000000000004), (3.0, 1.0, '20', 0.56976744186046513), (3.0, 1.5, '01', 0.34615384615384615), (3.0, 1.5, '02', 0.58333333333333337), (3.0, 1.5, '03', 0.59090909090909094), (3.0, 1.5, '04', 0.56000000000000005), (3.0, 1.5, '06', 0.546875), (3.0, 1.5, '07', 0.47499999999999998), (3.0, 1.5, '08', 0.5), (3.0, 1.5, '09', 0.47297297297297297), (3.0, 1.5, '10', 0.52941176470588236), (3.0, 1.5, '11', 0.53846153846153844), (3.0, 1.5, '12', 0.46721311475409838), (3.0, 1.5, '13', 0.39534883720930231), (3.0, 1.5, '14', 0.45744680851063829), (3.0, 1.5, '15', 0.48484848484848486), (3.0, 1.5, '16', 0.41025641025641024), (3.0, 1.5, '17', 0.64864864864864868), (3.0, 1.5, '18', 0.4375), (3.0, 1.5, '19', 0.53333333333333333), (3.0, 1.5, '20', 0.39534883720930231), (3.0, 2.0, '01', 0.65384615384615385), (3.0, 2.0, '02', 0.375), (3.0, 2.0, '03', 0.40909090909090912), (3.0, 2.0, '04', 0.57999999999999996), (3.0, 2.0, '06', 0.53125), (3.0, 2.0, '07', 0.55000000000000004), (3.0, 2.0, '08', 0.50624999999999998), (3.0, 2.0, '09', 0.56756756756756754), (3.0, 2.0, '10', 0.51470588235294112), (3.0, 2.0, '11', 0.58653846153846156), (3.0, 2.0, '12', 0.45901639344262296), (3.0, 2.0, '13', 0.43023255813953487), (3.0, 2.0, '14', 0.46808510638297873), (3.0, 2.0, '15', 0.48484848484848486), (3.0, 2.0, '16', 0.47435897435897434), (3.0, 2.0, '17', 0.44594594594594594), (3.0, 2.0, '18', 0.421875), (3.0, 2.0, '19', 0.43333333333333335), (3.0, 2.0, '20', 0.45348837209302323), (3.0, 2.5, '01', 0.57692307692307687), (3.0, 2.5, '02', 0.41666666666666669), (3.0, 2.5, '03', 0.45454545454545453), (3.0, 2.5, '04', 0.66000000000000003), (3.0, 2.5, '06', 0.53125), (3.0, 2.5, '07', 0.49166666666666664), (3.0, 2.5, '08', 0.51875000000000004), (3.0, 2.5, '09', 0.44594594594594594), (3.0, 2.5, '10', 0.5), (3.0, 2.5, '11', 0.54807692307692313), (3.0, 2.5, '12', 0.38524590163934425), (3.0, 2.5, '13', 0.40697674418604651), (3.0, 2.5, '14', 0.40425531914893614), (3.0, 2.5, '15', 0.46969696969696972), (3.0, 2.5, '16', 0.32051282051282054), (3.0, 2.5, '17', 0.41891891891891891), (3.0, 2.5, '18', 0.4375), (3.0, 2.5, '19', 0.48333333333333334), (3.0, 2.5, '20', 0.52325581395348841), (3.0, 3.0, '01', 0.65384615384615385), (3.0, 3.0, '02', 0.41666666666666669), (3.0, 3.0, '03', 0.54545454545454541), (3.0, 3.0, '04', 0.59999999999999998), (3.0, 3.0, '06', 0.578125), (3.0, 3.0, '07', 0.58333333333333337), (3.0, 3.0, '08', 0.41875000000000001), (3.0, 3.0, '09', 0.3783783783783784), (3.0, 3.0, '10', 0.48529411764705882), (3.0, 3.0, '11', 0.57692307692307687), (3.0, 3.0, '12', 0.45081967213114754), (3.0, 3.0, '13', 0.46511627906976744), (3.0, 3.0, '14', 0.36170212765957449), (3.0, 3.0, '15', 0.51515151515151514), (3.0, 3.0, '16', 0.48717948717948717), (3.0, 3.0, '17', 0.44594594594594594), (3.0, 3.0, '18', 0.421875), (3.0, 3.0, '19', 0.38333333333333336), (3.0, 3.0, '20', 0.45348837209302323), (3.0, 3.5, '01', 0.61538461538461542), (3.0, 3.5, '02', 0.5), (3.0, 3.5, '03', 0.5), (3.0, 3.5, '04', 0.69999999999999996), (3.0, 3.5, '06', 0.484375), (3.0, 3.5, '07', 0.5), (3.0, 3.5, '08', 0.41875000000000001), (3.0, 3.5, '09', 0.48648648648648651), (3.0, 3.5, '10', 0.5), (3.0, 3.5, '11', 0.56730769230769229), (3.0, 3.5, '12', 0.45901639344262296), (3.0, 3.5, '13', 0.48837209302325579), (3.0, 3.5, '14', 0.37234042553191488), (3.0, 3.5, '15', 0.43939393939393939), (3.0, 3.5, '16', 0.48717948717948717), (3.0, 3.5, '17', 0.48648648648648651), (3.0, 3.5, '18', 0.421875), (3.0, 3.5, '19', 0.38333333333333336), (3.0, 3.5, '20', 0.55813953488372092), (3.0, 4.0, '01', 0.53846153846153844), (3.0, 4.0, '02', 0.41666666666666669), (3.0, 4.0, '03', 0.54545454545454541), (3.0, 4.0, '04', 0.5), (3.0, 4.0, '06', 0.5), (3.0, 4.0, '07', 0.58333333333333337), (3.0, 4.0, '08', 0.41249999999999998), (3.0, 4.0, '09', 0.5), (3.0, 4.0, '10', 0.5), (3.0, 4.0, '11', 0.57692307692307687), (3.0, 4.0, '12', 0.49180327868852458), (3.0, 4.0, '13', 0.46511627906976744), (3.0, 4.0, '14', 0.38297872340425532), (3.0, 4.0, '15', 0.53030303030303028), (3.0, 4.0, '16', 0.46153846153846156), (3.0, 4.0, '17', 0.44594594594594594), (3.0, 4.0, '18', 0.421875), (3.0, 4.0, '19', 0.48333333333333334), (3.0, 4.0, '20', 0.58139534883720934), (3.0, 4.5, '01', 0.53846153846153844), (3.0, 4.5, '02', 0.5), (3.0, 4.5, '03', 0.63636363636363635), (3.0, 4.5, '04', 0.54000000000000004), (3.0, 4.5, '06', 0.515625), (3.0, 4.5, '07', 0.65833333333333333), (3.0, 4.5, '08', 0.39374999999999999), (3.0, 4.5, '09', 0.35135135135135137), (3.0, 4.5, '10', 0.5), (3.0, 4.5, '11', 0.54807692307692313), (3.0, 4.5, '12', 0.47540983606557374), (3.0, 4.5, '13', 0.38372093023255816), (3.0, 4.5, '14', 0.38297872340425532), (3.0, 4.5, '15', 0.46969696969696972), (3.0, 4.5, '16', 0.52564102564102566), (3.0, 4.5, '17', 0.41891891891891891), (3.0, 4.5, '18', 0.421875), (3.0, 4.5, '19', 0.5), (3.0, 4.5, '20', 0.56976744186046513), (3.5, 0.5, '01', 0.42307692307692307), (3.5, 0.5, '02', 0.5), (3.5, 0.5, '03', 0.5), (3.5, 0.5, '04', 0.71999999999999997), (3.5, 0.5, '06', 0.484375), (3.5, 0.5, '07', 0.40833333333333333), (3.5, 0.5, '08', 0.48749999999999999), (3.5, 0.5, '09', 0.55405405405405406), (3.5, 0.5, '10', 0.52941176470588236), (3.5, 0.5, '11', 0.53846153846153844), (3.5, 0.5, '12', 0.50819672131147542), (3.5, 0.5, '13', 0.52325581395348841), (3.5, 0.5, '14', 0.5), (3.5, 0.5, '15', 0.59090909090909094), (3.5, 0.5, '16', 0.55128205128205132), (3.5, 0.5, '17', 0.5), (3.5, 0.5, '18', 0.4375), (3.5, 0.5, '19', 0.51666666666666672), (3.5, 0.5, '20', 0.52325581395348841), (3.5, 1.0, '01', 0.38461538461538464), (3.5, 1.0, '02', 0.58333333333333337), (3.5, 1.0, '03', 0.40909090909090912), (3.5, 1.0, '04', 0.59999999999999998), (3.5, 1.0, '06', 0.59375), (3.5, 1.0, '07', 0.49166666666666664), (3.5, 1.0, '08', 0.51249999999999996), (3.5, 1.0, '09', 0.41891891891891891), (3.5, 1.0, '10', 0.51470588235294112), (3.5, 1.0, '11', 0.52884615384615385), (3.5, 1.0, '12', 0.54098360655737709), (3.5, 1.0, '13', 0.51162790697674421), (3.5, 1.0, '14', 0.53191489361702127), (3.5, 1.0, '15', 0.51515151515151514), (3.5, 1.0, '16', 0.4358974358974359), (3.5, 1.0, '17', 0.39189189189189189), (3.5, 1.0, '18', 0.453125), (3.5, 1.0, '19', 0.5), (3.5, 1.0, '20', 0.41860465116279072), (3.5, 1.5, '01', 0.46153846153846156), (3.5, 1.5, '02', 0.625), (3.5, 1.5, '03', 0.72727272727272729), (3.5, 1.5, '04', 0.62), (3.5, 1.5, '06', 0.546875), (3.5, 1.5, '07', 0.48333333333333334), (3.5, 1.5, '08', 0.51875000000000004), (3.5, 1.5, '09', 0.41891891891891891), (3.5, 1.5, '10', 0.48529411764705882), (3.5, 1.5, '11', 0.52884615384615385), (3.5, 1.5, '12', 0.54098360655737709), (3.5, 1.5, '13', 0.40697674418604651), (3.5, 1.5, '14', 0.51063829787234039), (3.5, 1.5, '15', 0.54545454545454541), (3.5, 1.5, '16', 0.37179487179487181), (3.5, 1.5, '17', 0.44594594594594594), (3.5, 1.5, '18', 0.421875), (3.5, 1.5, '19', 0.53333333333333333), (3.5, 1.5, '20', 0.55813953488372092), (3.5, 2.0, '01', 0.34615384615384615), (3.5, 2.0, '02', 0.54166666666666663), (3.5, 2.0, '03', 0.59090909090909094), (3.5, 2.0, '04', 0.40000000000000002), (3.5, 2.0, '06', 0.546875), (3.5, 2.0, '07', 0.55000000000000004), (3.5, 2.0, '08', 0.48749999999999999), (3.5, 2.0, '09', 0.35135135135135137), (3.5, 2.0, '10', 0.55882352941176472), (3.5, 2.0, '11', 0.54807692307692313), (3.5, 2.0, '12', 0.5901639344262295), (3.5, 2.0, '13', 0.5), (3.5, 2.0, '14', 0.48936170212765956), (3.5, 2.0, '15', 0.48484848484848486), (3.5, 2.0, '16', 0.37179487179487181), (3.5, 2.0, '17', 0.47297297297297297), (3.5, 2.0, '18', 0.484375), (3.5, 2.0, '19', 0.48333333333333334), (3.5, 2.0, '20', 0.48837209302325579), (3.5, 2.5, '01', 0.34615384615384615), (3.5, 2.5, '02', 0.5), (3.5, 2.5, '03', 0.54545454545454541), (3.5, 2.5, '04', 0.62), (3.5, 2.5, '06', 0.53125), (3.5, 2.5, '07', 0.54166666666666663), (3.5, 2.5, '08', 0.47499999999999998), (3.5, 2.5, '09', 0.48648648648648651), (3.5, 2.5, '10', 0.48529411764705882), (3.5, 2.5, '11', 0.55769230769230771), (3.5, 2.5, '12', 0.53278688524590168), (3.5, 2.5, '13', 0.46511627906976744), (3.5, 2.5, '14', 0.41489361702127658), (3.5, 2.5, '15', 0.56060606060606055), (3.5, 2.5, '16', 0.41025641025641024), (3.5, 2.5, '17', 0.51351351351351349), (3.5, 2.5, '18', 0.53125), (3.5, 2.5, '19', 0.46666666666666667), (3.5, 2.5, '20', 0.5), (3.5, 3.0, '01', 0.5), (3.5, 3.0, '02', 0.41666666666666669), (3.5, 3.0, '03', 0.63636363636363635), (3.5, 3.0, '04', 0.62), (3.5, 3.0, '06', 0.515625), (3.5, 3.0, '07', 0.51666666666666672), (3.5, 3.0, '08', 0.46250000000000002), (3.5, 3.0, '09', 0.41891891891891891), (3.5, 3.0, '10', 0.47058823529411764), (3.5, 3.0, '11', 0.54807692307692313), (3.5, 3.0, '12', 0.48360655737704916), (3.5, 3.0, '13', 0.47674418604651164), (3.5, 3.0, '14', 0.44680851063829785), (3.5, 3.0, '15', 0.56060606060606055), (3.5, 3.0, '16', 0.42307692307692307), (3.5, 3.0, '17', 0.44594594594594594), (3.5, 3.0, '18', 0.4375), (3.5, 3.0, '19', 0.5), (3.5, 3.0, '20', 0.48837209302325579), (3.5, 3.5, '01', 0.5), (3.5, 3.5, '02', 0.5), (3.5, 3.5, '03', 0.63636363636363635), (3.5, 3.5, '04', 0.56000000000000005), (3.5, 3.5, '06', 0.546875), (3.5, 3.5, '07', 0.45000000000000001), (3.5, 3.5, '08', 0.42499999999999999), (3.5, 3.5, '09', 0.39189189189189189), (3.5, 3.5, '10', 0.45588235294117646), (3.5, 3.5, '11', 0.55769230769230771), (3.5, 3.5, '12', 0.57377049180327866), (3.5, 3.5, '13', 0.41860465116279072), (3.5, 3.5, '14', 0.40425531914893614), (3.5, 3.5, '15', 0.46969696969696972), (3.5, 3.5, '16', 0.44871794871794873), (3.5, 3.5, '17', 0.43243243243243246), (3.5, 3.5, '18', 0.421875), (3.5, 3.5, '19', 0.51666666666666672), (3.5, 3.5, '20', 0.58139534883720934), (3.5, 4.0, '01', 0.46153846153846156), (3.5, 4.0, '02', 0.54166666666666663), (3.5, 4.0, '03', 0.54545454545454541), (3.5, 4.0, '04', 0.68000000000000005), (3.5, 4.0, '06', 0.59375), (3.5, 4.0, '07', 0.54166666666666663), (3.5, 4.0, '08', 0.47499999999999998), (3.5, 4.0, '09', 0.45945945945945948), (3.5, 4.0, '10', 0.45588235294117646), (3.5, 4.0, '11', 0.55769230769230771), (3.5, 4.0, '12', 0.54918032786885251), (3.5, 4.0, '13', 0.5), (3.5, 4.0, '14', 0.38297872340425532), (3.5, 4.0, '15', 0.53030303030303028), (3.5, 4.0, '16', 0.5), (3.5, 4.0, '17', 0.51351351351351349), (3.5, 4.0, '18', 0.4375), (3.5, 4.0, '19', 0.43333333333333335), (3.5, 4.0, '20', 0.43023255813953487), (3.5, 4.5, '01', 0.57692307692307687), (3.5, 4.5, '02', 0.5), (3.5, 4.5, '03', 0.63636363636363635), (3.5, 4.5, '04', 0.47999999999999998), (3.5, 4.5, '06', 0.578125), (3.5, 4.5, '07', 0.51666666666666672), (3.5, 4.5, '08', 0.49375000000000002), (3.5, 4.5, '09', 0.48648648648648651), (3.5, 4.5, '10', 0.48529411764705882), (3.5, 4.5, '11', 0.55769230769230771), (3.5, 4.5, '12', 0.53278688524590168), (3.5, 4.5, '13', 0.46511627906976744), (3.5, 4.5, '14', 0.40425531914893614), (3.5, 4.5, '15', 0.46969696969696972), (3.5, 4.5, '16', 0.5), (3.5, 4.5, '17', 0.47297297297297297), (3.5, 4.5, '18', 0.4375), (3.5, 4.5, '19', 0.51666666666666672), (3.5, 4.5, '20', 0.47674418604651164), (4.0, 0.5, '01', 0.46153846153846156), (4.0, 0.5, '02', 0.58333333333333337), (4.0, 0.5, '03', 0.5), (4.0, 0.5, '04', 0.66000000000000003), (4.0, 0.5, '06', 0.484375), (4.0, 0.5, '07', 0.55000000000000004), (4.0, 0.5, '08', 0.59375), (4.0, 0.5, '09', 0.58108108108108103), (4.0, 0.5, '10', 0.54411764705882348), (4.0, 0.5, '11', 0.46153846153846156), (4.0, 0.5, '12', 0.51639344262295084), (4.0, 0.5, '13', 0.47674418604651164), (4.0, 0.5, '14', 0.5957446808510638), (4.0, 0.5, '15', 0.59090909090909094), (4.0, 0.5, '16', 0.61538461538461542), (4.0, 0.5, '17', 0.55405405405405406), (4.0, 0.5, '18', 0.578125), (4.0, 0.5, '19', 0.46666666666666667), (4.0, 0.5, '20', 0.41860465116279072), (4.0, 1.0, '01', 0.53846153846153844), (4.0, 1.0, '02', 0.58333333333333337), (4.0, 1.0, '03', 0.40909090909090912), (4.0, 1.0, '04', 0.41999999999999998), (4.0, 1.0, '06', 0.46875), (4.0, 1.0, '07', 0.56666666666666665), (4.0, 1.0, '08', 0.63749999999999996), (4.0, 1.0, '09', 0.56756756756756754), (4.0, 1.0, '10', 0.47058823529411764), (4.0, 1.0, '11', 0.52884615384615385), (4.0, 1.0, '12', 0.46721311475409838), (4.0, 1.0, '13', 0.56976744186046513), (4.0, 1.0, '14', 0.52127659574468088), (4.0, 1.0, '15', 0.54545454545454541), (4.0, 1.0, '16', 0.55128205128205132), (4.0, 1.0, '17', 0.48648648648648651), (4.0, 1.0, '18', 0.34375), (4.0, 1.0, '19', 0.5), (4.0, 1.0, '20', 0.41860465116279072), (4.0, 1.5, '01', 0.53846153846153844), (4.0, 1.5, '02', 0.54166666666666663), (4.0, 1.5, '03', 0.5), (4.0, 1.5, '04', 0.38), (4.0, 1.5, '06', 0.328125), (4.0, 1.5, '07', 0.60833333333333328), (4.0, 1.5, '08', 0.60624999999999996), (4.0, 1.5, '09', 0.6216216216216216), (4.0, 1.5, '10', 0.4264705882352941), (4.0, 1.5, '11', 0.49038461538461536), (4.0, 1.5, '12', 0.51639344262295084), (4.0, 1.5, '13', 0.55813953488372092), (4.0, 1.5, '14', 0.46808510638297873), (4.0, 1.5, '15', 0.54545454545454541), (4.0, 1.5, '16', 0.4358974358974359), (4.0, 1.5, '17', 0.60810810810810811), (4.0, 1.5, '18', 0.40625), (4.0, 1.5, '19', 0.55000000000000004), (4.0, 1.5, '20', 0.47674418604651164), (4.0, 2.0, '01', 0.65384615384615385), (4.0, 2.0, '02', 0.45833333333333331), (4.0, 2.0, '03', 0.45454545454545453), (4.0, 2.0, '04', 0.78000000000000003), (4.0, 2.0, '06', 0.390625), (4.0, 2.0, '07', 0.6166666666666667), (4.0, 2.0, '08', 0.55000000000000004), (4.0, 2.0, '09', 0.56756756756756754), (4.0, 2.0, '10', 0.45588235294117646), (4.0, 2.0, '11', 0.51923076923076927), (4.0, 2.0, '12', 0.48360655737704916), (4.0, 2.0, '13', 0.55813953488372092), (4.0, 2.0, '14', 0.45744680851063829), (4.0, 2.0, '15', 0.51515151515151514), (4.0, 2.0, '16', 0.53846153846153844), (4.0, 2.0, '17', 0.44594594594594594), (4.0, 2.0, '18', 0.390625), (4.0, 2.0, '19', 0.51666666666666672), (4.0, 2.0, '20', 0.44186046511627908), (4.0, 2.5, '01', 0.73076923076923073), (4.0, 2.5, '02', 0.5), (4.0, 2.5, '03', 0.31818181818181818), (4.0, 2.5, '04', 0.80000000000000004), (4.0, 2.5, '06', 0.28125), (4.0, 2.5, '07', 0.5), (4.0, 2.5, '08', 0.48125000000000001), (4.0, 2.5, '09', 0.51351351351351349), (4.0, 2.5, '10', 0.4264705882352941), (4.0, 2.5, '11', 0.55769230769230771), (4.0, 2.5, '12', 0.52459016393442626), (4.0, 2.5, '13', 0.5), (4.0, 2.5, '14', 0.45744680851063829), (4.0, 2.5, '15', 0.46969696969696972), (4.0, 2.5, '16', 0.47435897435897434), (4.0, 2.5, '17', 0.47297297297297297), (4.0, 2.5, '18', 0.484375), (4.0, 2.5, '19', 0.48333333333333334), (4.0, 2.5, '20', 0.43023255813953487), (4.0, 3.0, '01', 0.80769230769230771), (4.0, 3.0, '02', 0.41666666666666669), (4.0, 3.0, '03', 0.5), (4.0, 3.0, '04', 0.5), (4.0, 3.0, '06', 0.28125), (4.0, 3.0, '07', 0.5), (4.0, 3.0, '08', 0.51249999999999996), (4.0, 3.0, '09', 0.48648648648648651), (4.0, 3.0, '10', 0.44117647058823528), (4.0, 3.0, '11', 0.52884615384615385), (4.0, 3.0, '12', 0.5901639344262295), (4.0, 3.0, '13', 0.45348837209302323), (4.0, 3.0, '14', 0.46808510638297873), (4.0, 3.0, '15', 0.48484848484848486), (4.0, 3.0, '16', 0.44871794871794873), (4.0, 3.0, '17', 0.40540540540540543), (4.0, 3.0, '18', 0.46875), (4.0, 3.0, '19', 0.5), (4.0, 3.0, '20', 0.48837209302325579), (4.0, 3.5, '01', 0.69230769230769229), (4.0, 3.5, '02', 0.41666666666666669), (4.0, 3.5, '03', 0.5), (4.0, 3.5, '04', 0.54000000000000004), (4.0, 3.5, '06', 0.328125), (4.0, 3.5, '07', 0.55000000000000004), (4.0, 3.5, '08', 0.51249999999999996), (4.0, 3.5, '09', 0.48648648648648651), (4.0, 3.5, '10', 0.47058823529411764), (4.0, 3.5, '11', 0.53846153846153844), (4.0, 3.5, '12', 0.51639344262295084), (4.0, 3.5, '13', 0.43023255813953487), (4.0, 3.5, '14', 0.41489361702127658), (4.0, 3.5, '15', 0.45454545454545453), (4.0, 3.5, '16', 0.53846153846153844), (4.0, 3.5, '17', 0.43243243243243246), (4.0, 3.5, '18', 0.484375), (4.0, 3.5, '19', 0.5), (4.0, 3.5, '20', 0.56976744186046513), (4.0, 4.0, '01', 0.65384615384615385), (4.0, 4.0, '02', 0.5), (4.0, 4.0, '03', 0.45454545454545453), (4.0, 4.0, '04', 0.46000000000000002), (4.0, 4.0, '06', 0.265625), (4.0, 4.0, '07', 0.59166666666666667), (4.0, 4.0, '08', 0.54374999999999996), (4.0, 4.0, '09', 0.44594594594594594), (4.0, 4.0, '10', 0.45588235294117646), (4.0, 4.0, '11', 0.51923076923076927), (4.0, 4.0, '12', 0.50819672131147542), (4.0, 4.0, '13', 0.47674418604651164), (4.0, 4.0, '14', 0.47872340425531917), (4.0, 4.0, '15', 0.43939393939393939), (4.0, 4.0, '16', 0.51282051282051277), (4.0, 4.0, '17', 0.43243243243243246), (4.0, 4.0, '18', 0.484375), (4.0, 4.0, '19', 0.51666666666666672), (4.0, 4.0, '20', 0.48837209302325579), (4.0, 4.5, '01', 0.5), (4.0, 4.5, '02', 0.5), (4.0, 4.5, '03', 0.36363636363636365), (4.0, 4.5, '04', 0.41999999999999998), (4.0, 4.5, '06', 0.296875), (4.0, 4.5, '07', 0.59166666666666667), (4.0, 4.5, '08', 0.54374999999999996), (4.0, 4.5, '09', 0.5), (4.0, 4.5, '10', 0.4264705882352941), (4.0, 4.5, '11', 0.51923076923076927), (4.0, 4.5, '12', 0.51639344262295084), (4.0, 4.5, '13', 0.53488372093023251), (4.0, 4.5, '14', 0.5), (4.0, 4.5, '15', 0.45454545454545453), (4.0, 4.5, '16', 0.51282051282051277), (4.0, 4.5, '17', 0.5), (4.0, 4.5, '18', 0.46875), (4.0, 4.5, '19', 0.48333333333333334), (4.0, 4.5, '20', 0.52325581395348841)]
print "Subject ID | Testing acc on this subject | optimal parameters\n"

# For each outer fold, choose best params (ed, mel) based on validation score and record corresponding test_acc
test_results = []
for SID in SIDs:
    # take validation results from outer fold SID, taking ed, mel, C, val_acc
    vr = np.array([(r[0],r[1],r[2],r[4]) for r in val_scores if r[3] == SID])     
    i = np.argmax(vr[:,3]) # index in array vr where val_score is maximal
    best_ed, best_mel, best_C, _ = vr[i]
    test_results.extend([r[3] for r in test_scores if r[0] == best_ed and r[1] == best_mel and r[2] == SID])
#     print len([r[3] for r in test_scores if r[0] == best_ed and r[1] == best_mel and r[2] == SID])
    print SID, ": ", test_results[-1], vr[i][0:3]
print "\nMean testing accracy (over all subjects): ", np.mean(test_results), "+/-", np.std(test_results)

Subject ID | Testing acc on this subject | optimal parameters

01 :  0.461538461538 [ 4.      0.5     0.0625]
02 :  0.541666666667 [ 4.      1.5     0.0625]
03 :  0.590909090909 [ 3.  1.  4.]
04 :  0.66 [  4.00000000e+00   5.00000000e-01   1.95312500e-03]
06 :  0.640625 [ 3.     1.     0.125]
07 :  0.516666666667 [ 3.   1.   0.5]
08 :  0.4875 [ 1.     2.5    0.125]
09 :  0.445945945946 [  1.50000000e+00   1.50000000e+00   9.76562500e-04]
10 :  0.544117647059 [ 3.       1.       0.03125]
11 :  0.490384615385 [  4.00000000e+00   1.50000000e+00   1.95312500e-03]
12 :  0.532786885246 [  2.5   3.   16. ]
13 :  0.56976744186 [ 3.   1.   0.5]
14 :  0.340425531915 [ 1.       2.       0.03125]
15 :  0.454545454545 [ 1.    2.5   0.25]
16 :  0.474358974359 [ 1.5         1.5         0.00195312]
17 :  0.554054054054 [  3.00000000e+00   1.00000000e+00   1.95312500e-03]
18 :  0.53125 [ 3.  1.  1.]
19 :  0.466666666667 [ 4.        0.5       0.015625]
20 :  0.488372093023 [ 1.    1.    0.25]

Mean test